In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re

from konlpy.tag import *

In [2]:
from konlpy.tag import Mecab

m = Mecab(dicpath = 'C:/mecab/mecab-ko-dic')
m.nouns('A타입도 좋고, PD충전 되는 것도 좋아요. C타입 여러개 있어서 좋습니다')

['A타입', 'PD충전', '것', 'C타입', '개']

In [3]:
df = pd.read_parquet('C:/Users/songy/Desktop/Data/sentiment_data/무선청소기_20240807_16_05_12.parquet')
df.head(3)

,brand_name,product_name,write_dt_lst,rating_lst,content_lst
0,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,흡입력이 매우 좋습니다.
1,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,"생각보다 작았지만 흡입력이 좋아서 만족합니다 집에서 간단하게 쓸 용도, 차량용으로 ..."
2,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,소음은 좀 있지만 생각보다는 바람이 세서 좋아요


In [4]:
len(df)

24283

In [5]:
df.drop_duplicates(inplace=True)
len(df)

24218

In [6]:
# hanspell 설치된 경로 
!pip show hanspell

Name: hanspell
Version: 0.1
Summary: editing hanspell package (for testing)
Home-page: https://github.com/Seokhyeon-Park/hanspell.git
Author: Seokbong
Author-email: dfbgq346q@gmail.com
License: 
Location: c:\users\songy\anaconda\envs\mecab\lib\site-packages
Requires: requests
Required-by: 


In [7]:
from hanspell import spell_checker

text = "왜 안되냐구 이거 바꿔 외않되"

spelled_text = spell_checker.check(text)
print(spelled_text)

Checked(result=True, original='왜 안되냐구 이거 바꿔 외않되', checked='왜 안되냐고 이거 바꿔 왜 안돼', errors=2, words=OrderedDict([('왜', 0), ('안되냐고', 0), ('이거', 0), ('바꿔', 0), ('안돼', 0)]), time=0.057497501373291016)


In [8]:
df.head()

,brand_name,product_name,write_dt_lst,rating_lst,content_lst
0,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,흡입력이 매우 좋습니다.
1,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,"생각보다 작았지만 흡입력이 좋아서 만족합니다 집에서 간단하게 쓸 용도, 차량용으로 ..."
2,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,소음은 좀 있지만 생각보다는 바람이 세서 좋아요
3,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,머리말린 후 바닥에 떨어진 머리카락 정리하는데 좋아요!
4,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,작은 사이즈에 흡입력도 좋아서 편리하게 잘 사용하고 있습니다.


In [9]:
from hanspell import spell_checker
import xml.etree.ElementTree as ET

# review_lst = review_df_1['spacing_review'].to_list()

spelled_checked_text = []
for lst in df['content_lst']:
    try:
        result = spell_checker.check(lst)
        spelled_text = result.checked
    except ET.ParseError:
        spelled_text = lst  # 오류 발생 시 원래 텍스트 사용
    except Exception as e:
        spelled_text = lst  # 기타 예외 발생 시 원래 텍스트 사용
        print(f"Error processing text: {lst}\nException: {e}")
    
    spelled_checked_text.append(spelled_text)

#df['spelled_checked_text'] = spelled_checked_text
df['content_lst'] = spelled_checked_text

Error processing text: 제가 쓰는거 보더니 이모가 작은청소기가 강력하고 좋다며 사달라고 해서 사드렸는데요~~대만족이랍니다ㅎㅎ 충전해놓으면 오래가고 참좋아요^^
Exception: Expecting value: line 1 column 1 (char 0)
Error processing text: 기존 청소기 교체시기가 된것같아 청소기 폭풍써치중 전에 세컨청소기로 사용하던 캐치웰에서 대형사고를 내셨더라구요 자동먼지통비움 매직타워라니요.....그것도 20만원대......,,실화?? 대기업제품만 보다가 보니 가격일단 거의 1/4정도밖에 안되고 성능도 야무지고 디자인도 손색없네요 행운인지 청소기보고있으니 3일후에 라방예정이 있길래 기다렸다가 혜택많이 받아서 주문했구요 저렴한가격에 미니청소기에 물걸레까지 받았습니다 바로 다다음날도착해서 받자마자 청소기 돌렸습니다 성능좋아요 기본2단으로 돌렸는데도 고양이털 모래 충분히 다 잘빨아들이네요 상품평 이렇게 길게써본거 처음ㅋㅋㅋ너무좋아서 돈번느낌이에요 아 단점도 있긴한데 타워가 아무래도 무게감이 없다보니 꽂을때나 뺄때 타워를 잡고 해야된다는 소소한 단점이 있구요 이점은 보완되면 더 좋을것같습니다 그리고 두번째는 필터등 소모품이 사이트에서도 구매하기가 어렵다고들 하시는데 네이버스토에서도 간편하게 구입할수있으면 좋을것같아요
Exception: Expecting value: line 1 column 1 (char 0)
Error processing text: 빠르게 배송왔구요 개봉스티커는 뜯어져있던데 반품이든 검수든 뭐있겠지요? 생각보다 무겁네요 다이ㅅ,차이ㅅ,쿠ㅋ, 등 5가지 무선청소기 써봤는데 제일 무거운것같아요 손잡이쪽도 그렇고 헤드브러쉬도 크기가 큰만큼 무겁. 헤드브러쉬는 하도 덜덜거리기에 고객센터 문의했더니 브러쉬에 모터있어서 회전력이 좋아서 그런거라고 정상이라하네요 글쎄요ᆢ손잡이까지 바닥을 치는 드드드 느낌이 전해진다는건 사용자입장에선 고장확률도 높아지지 않을까 싶어요. 모터가있고 회전력이 빠른만큼 보강을 

In [10]:
def upper_text(text):
    if pd.notnull(text):  # 텍스트가 Null이 아닌 경우에만 변환
        return text.upper()
    return text  # Null 값은 그대로 반환

#df['spelled_checked_text'] = df['spelled_checked_text'].apply(upper_text)
df['content_lst'] = df['content_lst'].apply(upper_text)

In [11]:
def replace_text(text):
  text = str(text)
  text = text.replace('쌈ㅎ','저렴해요')
  text = text.replace('안되는...','안됩니다')
  text = text.replace('ㅎ','')
  text = text.replace('ㅋ','')
  text = text.replace('루♡','')
  text = text.replace('♡','')
  text = text.replace('굳','굿')
  text = text.replace('ㅜ','')
  text = text.replace('중입니다)','중 입니다')
  text = text.replace('하나에요','하나예요')
  text = text.replace('있었어요','있어요')
  text = text.replace('욥','요')
  text = text.replace('오겠씀다~~ㅎㅎ/','오겠습니다')
  text = text.replace('후에~','')
  text = text.replace('개용','개요')
  text = text.replace('ㅅ','')
  text = text.replace('오ㅡ','')
  text = text.replace('여행가고싶어지네요ㅋㅋ','여행 가고 싶어지네요')
  text = text.replace('좋습니다!!!!','좋습니다')
  text = text.replace('구매확정해요','구매 확정해요')
  text = text.replace('완죤빨라요ㅋㅋ','완전 빨라요')
  text = text.replace('ㅡ','')
  text = text.replace('귀요미에요','귀여워요')
  text = text.replace('최고여요','최고예요')
  text = text.replace('사랑하여','사랑해요')
  text = text.replace('배송되었습니다','배송 되었습니다')
  text = text.replace('좋아요ㅔ','좋아요')
  text = text.replace('아니에요.','아니예요')
  text = text.replace('구매확정했습니다','구매 확정했습니다')
  text = text.replace('bbb','굿')
  text = text.replace('쓸ㄲ너ㅣㅇ르','')
  text = text.replace('같고요','같아요')
  text = text.replace('같네요','같아요')
  text = text.replace('씁니다','습니다')
  text = text.replace('넘좋와여','너무 좋아요')
  text = text.replace('굿입니다','굿')
  text = text.replace('목배게최고에요','목배게 최고예요')
  text = text.replace('감동이에요','감동이예요')
  text = text.replace('아이디어템','아이디어 탬')
  text = text.replace(':D','')
  text = text.replace('감사핮니다','감사합니다')
  text = text.replace('템','탬')
  text = text.replace('든든 합니다','든든합니다')
  text = text.replace('받아보았어요','받아봤어요')
  text = text.replace('가타용','같아요')
  text = text.replace('바랍니아','바랍니다')
  text = text.replace('사업번창하세요','사업 번창하세요')
  text = text.replace('쓸겠습니다','쓰겠습니다')
  text = text.replace('구매완료입니다','구매완료')
  text = text.replace('짱빠름','짱 빠름')
  text = text.replace('됩니닽','됩니다')
  text = text.replace('닙니다ㅛ','닙니다')
  text = text.replace('잘한듯해요','잘한듯')
  text = text.replace('가세여','가세요')
  text = text.replace('잘샀다템','잘샀어요')
  text = text.replace('좋아요좋아요좋오ㅓㅗㅇ좋아요','좋아요 좋아요')
  text = text.replace('써봐겠ㅆ네요','써봐야겠네요')
  text = text.replace('만족이에요','만족이예요')
  text = text.replace('아까워요.','아까워요')
  text = text.replace('ajjqjahaja','')
  text = text.replace('잚슬겡ㅎ','잘쓸게요')
  text = text.replace('잘했아요','잘했어요')
  text = text.replace('괜찬ㅍ아요','괜찮아요')
  text = text.replace('예쁨','예뻐요')
  text = text.replace('편하쥬','편해요')
  text = text.replace('감사함다','감사합니다')
  text = text.replace('핳핳','')
  text = text.replace('굿입니다','굿')
  text = text.replace('으흐흐','')
  text = text.replace('빨라요ㅗ','빨라요')
  text = text.replace('이븜','이쁨')
  text = text.replace('ㄹㅇ','')
  text = text.replace('되매요','되네요')
  text = text.replace('믿어보겠음다','믿어보겠습니다')
  text = text.replace('아니에요','아니예요')
  text = text.replace('만족합니다.ㅣ','만족합니다')
  text = text.replace('되고요','됩니다')
  text = text.replace('딱입니딘','딱입니다')
  text = text.replace('기대되어요','기대됩니다')
  text = text.replace('왔습디다','왔습니다')
  text = text.replace('맀어용','랐어요')
  text = text.replace('강강히','')
  text = text.replace('만족이여','만족이요')
  text = text.replace('사용했니다.','사용했습니다')
  text = text.replace('충전빨리되구죻아요','충전 빨리되고 좋아요')
  text = text.replace('잘됩니다잉','잘됩니다')
  text = text.replace('사용했답니다','사용했어요')
  text = text.replace('가타요','같아요')
  text = text.replace('최곱니다','최고입니다')
  text = text.replace('빠르네요','빠릅니다')
  text = text.replace('기릉','기를')
  text = text.replace('받았습니아','받았습니다')
  text = text.replace('ok.','ok')
  text = text.replace('가져감','가져갑니다')
  text = text.replace('익니다','입니다')
  text = text.replace('수용되오ㅡ','수용되요')
  text = text.replace('맨죡','만족')
  text = text.replace('ㄹ','')
  text = text.replace('구매예정이에요','구매 예정이예요')
  text = text.replace('좋은듓','좋은듯')
  text = text.replace('매우저롐하게잘구매했어요감사하네여','매우 저렴하게 잘 구매했어요 감사해요')
  text = text.replace('만적합니다','만족합니다')
  text = text.replace('ㅈ번창하세요','번창하세요')
  text = text.replace('전이에요','전이예요')
  text = text.replace('조어요','좋아요')
  text = text.replace('젛아요','좋아요')
  text = text.replace('됩니더','됩니다')
  text = text.replace('굿입디다','굿')
  text = text.replace('갘아어요','')
  text = text.replace('ㄷ,','')
  text = text.replace('잘되겠쥬','잘되겠죠')
  text = text.replace('쓰게습니도','쓰겠습니다')
  text = text.replace('하고요','하고')
  text = text.replace('저뇽가방도있네요','전용가방도 있네요')
  text = text.replace('sjsjsjjsjs','')
  text = text.replace('좋겠구먼요','좋겠네요')
  text = text.replace('좋아욫','좋아요')
  text = text.replace('반퓸합니다','반품합니다')
  text = text.replace('꿋꿋입니다','굿')
  text = text.replace('잘씅게용','잘쓸게요')
  text = text.replace('땡큐네요','감사합니다')
  text = text.replace('베릐굿','굿')
  text = text.replace('효오','')
  text = text.replace('☆','')
  text = text.replace('ᄒ','')
  text = text.replace('썼어ᄉ','썼어요')
  text = text.replace('사용시뉴용할듯욕','사용시 유용할 듯')
  text = text.replace('있아요','있어요')
  text = text.replace('아다아다아','')
  text = text.replace('아다 아다 아','')
  text = text.replace('감사합니다아','감사합니다')
  text = text.replace('goood','굿')
  text = text.replace('싱픔굿배소오쟈웁','상품 굿 배송')
  text = text.replace('쓸겡','쓸게요')
  text = text.replace('굿국국숙','굿굿굿')
  text = text.replace('졸아요','좋아요')
  text = text.replace('USC','USB')
  text = text.replace('A S','AS')
  text = text.replace(' ㅐ외','해외')
  text = text.replace('ᄆ','')
  text = text.replace('ᅳᄋ ᄋ','')
  text = text.replace('배슝','배송')
  text = text.replace('ᅲ','')
  text = text.replace('쓸ᄁ너ᅵᄋ르','쓸게요')
  text = text.replace(' ♥','')
  text = text.replace('ᄏ','')
  text = text.replace('강 추','강추')
  text = text.replace('★','')
  text = text.replace('■','')
  
  # text = text.replace('USB', '유에스비')
  text = text.replace('필수 템', '필수템')
  text = text.replace('멀티 어댑터', '멀티어댑터')
  text = text.replace('EU 타입', 'EU타입')
  text = text.replace('느리편', '느린편')
  text = text.replace('요거', '이거')
  text = text.replace('여러 개', '여러개') 
  text = text.replace('내구 성', '내구성')
  text = text.replace('걱정 없이', '걱정없이')
  text = text.replace('부담 없는', '부담없는')
  text = text.replace('내구 성', '내구성')
  text = text.replace('선망', '선만')
  text = text.replace('필요 없음', '필요없음')
  text = text.replace('여향', '여행')
  text = text.replace('있렁서', '있어서')
  text = text.replace('필요 없음', '필요없음')
  text = text.replace('봏아요', '좋아요')
  text = text.replace('죠아용', '좋아요')
  text = text.replace('필요 없음', '필요없음')
  text = text.replace('좋았늡', '좋았습')
  text = text.replace('개벼운', '가벼운')
  text = text.replace('티스트', '테스트')

  text = text.replace('보관 백', '보관백')
  text = text.replace('TYPE', '타입')
  text = text.replace('C TO C', 'C타입')
  text = text.replace('CTOC', 'C타입')
  text = text.replace('C2C', 'C타입')
  text = text.replace('USB-C', 'USB포트 C타입')
  text = text.replace('USB-A', 'USB포트 A타입')
  text = text.replace('C TYPE', 'C타입')
  text = text.replace('C 타입','C타입')
  text = text.replace('씨타입','C타입')
  text = text.replace('씨 타입', 'C타입')
  text = text.replace('A 타입','A타입')
  text = text.replace('에이타입','A타입')
  text = text.replace('에이 타입', 'A타입')
  text = text.replace('EU 타입', 'EU타입')
  text = text.replace('PD 충전','PD충전')
  text = text.replace('PD 잭', 'PD잭')
  text = text.replace('급 속','급속')
  text = text.replace('포드','포트')
  text = text.replace('포드만','포트')
  text = text.replace('포드리','포트')
  text = text.replace('핸폰','핸드폰')
  text = text.replace('휴대폰', '핸드폰')
  text = text.replace('휴대전화', '핸드폰')
  text = text.replace('11볼트', '110V')
  text = text.replace('22볼트', '220V')
  text = text.replace('11V', '110V')
  text = text.replace('22V', '220V')
  # text = text.replace('35W', '35와트')
  # text = text.replace('45W', '45와트')
  # text = text.replace('65W', '65와트')
  text = text.replace('포터', '포트')
  # text = text.replace('USB', 'USB포트') 
  text = text.replace('USB 포트', 'USB포트')
  text = text.replace('동시 충전', '동시충전')
  text = text.replace('고속 충전', '고속충전')
  text = text.replace('동시 이용', '동시이용')
  text = text.replace('동시 사용', '동시사용')
  text = text.replace('여러 나라', '여러나라')
  text = text.replace('코넥터', '커넥터')
  # 1개짜리는 제외
  return text

In [12]:
df['content_lst'] = df['content_lst'].apply(replace_text)

In [13]:
df.head()

,brand_name,product_name,write_dt_lst,rating_lst,content_lst
0,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,흡입력이 매우 좋습니다.
1,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,"생각보다 작았지만 흡입력이 좋아서 만족합니다 집에서 간단하게 쓸 용도, 차량용으로 ..."
2,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,소음은 좀 있지만 생각보다는 바람이 세서 좋아요
3,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,머리 말린 후 바닥에 떨어진 머리카락 정리하는데 좋아요!
4,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,작은 사이즈에 흡입력도 좋아서 편리하게 잘 사용하고 있습니다.


kss하고

In [14]:
# 한국어 문장 분리기
#!pip install kss

In [15]:
import kss
from kss import split_sentences

def kss_sentence_split(text):
    return kss.split_sentences(text)

df['kss_split_text'] = df['content_lst'].apply(kss_sentence_split)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



In [16]:
df.head()

,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text
0,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,흡입력이 매우 좋습니다.,[흡입력이 매우 좋습니다.]
1,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,"생각보다 작았지만 흡입력이 좋아서 만족합니다 집에서 간단하게 쓸 용도, 차량용으로 ...","[생각보다 작았지만 흡입력이 좋아서 만족합니다, 집에서 간단하게 쓸 용도, 차량용으..."
2,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,소음은 좀 있지만 생각보다는 바람이 세서 좋아요,[소음은 좀 있지만 생각보다는 바람이 세서 좋아요]
3,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,머리 말린 후 바닥에 떨어진 머리카락 정리하는데 좋아요!,[머리 말린 후 바닥에 떨어진 머리카락 정리하는데 좋아요!]
4,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,작은 사이즈에 흡입력도 좋아서 편리하게 잘 사용하고 있습니다.,[작은 사이즈에 흡입력도 좋아서 편리하게 잘 사용하고 있습니다.]


In [18]:
# 여러 단어 기준으로 문자열 분리하는 함수 정의
def split_by_multiple_words(text, words):
    parts = [text]
    for word in words:
        new_parts = []
        for part in parts:
            new_parts.extend(split_by_word(part, word))
        parts = new_parts
    return parts

# 단어 기준으로 문자열 분리하는 기본 함수 정의
def split_by_word(text, word):
    parts = []
    start = 0
    while True:
        index = text.find(word, start)
        if index == -1:
            break
        parts.append(text[start:index + len(word)])
        start = index + len(word)
    parts.append(text[start:])
    return parts

# 여러 단어 기준으로 분리할 단어 리스트 정의
words_to_split = ['빠르고', '스럽고', ',']

# split_text 열을 분리하여 새로운 열 생성
df2 = df.explode(['kss_split_text'])
#df2['all_split_text'] = df2['kss_split_text'].apply(lambda x: split_by_multiple_words(x, words_to_split))
df2['kss_split_text'] = df2['kss_split_text'].apply(lambda x: split_by_multiple_words(str(x), words_to_split))
df2 = df2.explode(['kss_split_text'])
df2.head(3)

,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text
0,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,흡입력이 매우 좋습니다.,흡입력이 매우 좋습니다.
1,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,"생각보다 작았지만 흡입력이 좋아서 만족합니다 집에서 간단하게 쓸 용도, 차량용으로 ...",생각보다 작았지만 흡입력이 좋아서 만족합니다
1,미니센,미니센 무선 미니 청소기 핸디 소형 진공 원룸 머리카락 MNC-100,20240807,5,"생각보다 작았지만 흡입력이 좋아서 만족합니다 집에서 간단하게 쓸 용도, 차량용으로 ...","집에서 간단하게 쓸 용도,"


In [19]:
df2.to_csv('C:/Users/songy/Desktop/Data/sentiment_data/wireless_total.csv',encoding='utf-8-sig')

단순리뷰

In [4]:
df2 = pd.read_csv('C:/Users/songy/Desktop/Data/sentiment_data/wireless_total.csv')

In [5]:
df2.head(3)

,Unnamed: 0,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text
0,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,여행 가서 잘 썼습니다~
1,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,유럽용은 안 써도 될 듯 ㅠ
2,1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,여행에서 사용하려고요


In [ ]:
#### 위 함수 강화 버전 (고유 단어 개수 기반 추가 필터링)

def text_again_dev(text):
  words = text.split()

  # 중복 단어 케이스 (True: 중복 없음)
  result1 = (len(words)==len(set(words)))

  # 연속 단어 케이스
  result2 = 0
  for i in range(1, len(words)):
    if words[i] == words[i-1]:
      result2 = 1
    else:
      result2 = 0

  # 두 케이스가 모두 만족하는 경우 1 반환, 아니면 0 반환 (중복 & 연속)
  result3 = 0
  if (str(result1)=='False') & (result2==1):
    result3 = 1
  else:
    result3 = 0

  #### 세번째 케이스 (set 개수)
  set_len = len(set(words))
  if (result3 == 1) & (set_len <= 3):
    return 1
  else :
    return 0

df['repeated_final'] = df['spelled_checked_text'].apply(text_again_dev)
df.head(3)

In [ ]:
#product = pd.read_csv('C:/Users/songy/Desktop/Data/product.csv', index_col=0)
#product

In [ ]:
def upper_text(text):
    text = text.upper()
    return text

In [ ]:
product['add_rule_based_split_review'] = product['add_rule_based_split_review'].apply(upper_text)

In [ ]:
def replace_text(text):
  text = str(text)
  text = text.replace('쌈ㅎ','저렴해요')
  text = text.replace('안되는...','안됩니다')
  text = text.replace('ㅎ','')
  text = text.replace('ㅋ','')
  text = text.replace('루♡','')
  text = text.replace('♡','')
  text = text.replace('굳','굿')
  text = text.replace('ㅜ','')
  text = text.replace('중입니다)','중 입니다')
  text = text.replace('하나에요','하나예요')
  text = text.replace('있었어요','있어요')
  text = text.replace('욥','요')
  text = text.replace('오겠씀다~~ㅎㅎ/','오겠습니다')
  text = text.replace('후에~','')
  text = text.replace('개용','개요')
  text = text.replace('ㅅ','')
  text = text.replace('오ㅡ','')
  text = text.replace('여행가고싶어지네요ㅋㅋ','여행 가고 싶어지네요')
  text = text.replace('좋습니다!!!!','좋습니다')
  text = text.replace('구매확정해요','구매 확정해요')
  text = text.replace('완죤빨라요ㅋㅋ','완전 빨라요')
  text = text.replace('ㅡ','')
  text = text.replace('귀요미에요','귀여워요')
  text = text.replace('최고여요','최고예요')
  text = text.replace('사랑하여','사랑해요')
  text = text.replace('배송되었습니다','배송 되었습니다')
  text = text.replace('좋아요ㅔ','좋아요')
  text = text.replace('아니에요.','아니예요')
  text = text.replace('구매확정했습니다','구매 확정했습니다')
  text = text.replace('bbb','굿')
  text = text.replace('쓸ㄲ너ㅣㅇ르','')
  text = text.replace('같고요','같아요')
  text = text.replace('같네요','같아요')
  text = text.replace('씁니다','습니다')
  text = text.replace('넘좋와여','너무 좋아요')
  text = text.replace('굿입니다','굿')
  text = text.replace('목배게최고에요','목배게 최고예요')
  text = text.replace('감동이에요','감동이예요')
  text = text.replace('아이디어템','아이디어 탬')
  text = text.replace(':D','')
  text = text.replace('감사핮니다','감사합니다')
  text = text.replace('템','탬')
  text = text.replace('든든 합니다','든든합니다')
  text = text.replace('받아보았어요','받아봤어요')
  text = text.replace('가타용','같아요')
  text = text.replace('바랍니아','바랍니다')
  text = text.replace('사업번창하세요','사업 번창하세요')
  text = text.replace('쓸겠습니다','쓰겠습니다')
  text = text.replace('구매완료입니다','구매완료')
  text = text.replace('짱빠름','짱 빠름')
  text = text.replace('됩니닽','됩니다')
  text = text.replace('닙니다ㅛ','닙니다')
  text = text.replace('잘한듯해요','잘한듯')
  text = text.replace('가세여','가세요')
  text = text.replace('잘샀다템','잘샀어요')
  text = text.replace('좋아요좋아요좋오ㅓㅗㅇ좋아요','좋아요 좋아요')
  text = text.replace('써봐겠ㅆ네요','써봐야겠네요')
  text = text.replace('만족이에요','만족이예요')
  text = text.replace('아까워요.','아까워요')
  text = text.replace('ajjqjahaja','')
  text = text.replace('잚슬겡ㅎ','잘쓸게요')
  text = text.replace('잘했아요','잘했어요')
  text = text.replace('괜찬ㅍ아요','괜찮아요')
  text = text.replace('예쁨','예뻐요')
  text = text.replace('편하쥬','편해요')
  text = text.replace('감사함다','감사합니다')
  text = text.replace('핳핳','')
  text = text.replace('굿입니다','굿')
  text = text.replace('으흐흐','')
  text = text.replace('빨라요ㅗ','빨라요')
  text = text.replace('이븜','이쁨')
  text = text.replace('ㄹㅇ','')
  text = text.replace('되매요','되네요')
  text = text.replace('믿어보겠음다','믿어보겠습니다')
  text = text.replace('아니에요','아니예요')
  text = text.replace('만족합니다.ㅣ','만족합니다')
  text = text.replace('되고요','됩니다')
  text = text.replace('딱입니딘','딱입니다')
  text = text.replace('기대되어요','기대됩니다')
  text = text.replace('왔습디다','왔습니다')
  text = text.replace('맀어용','랐어요')
  text = text.replace('강강히','')
  text = text.replace('만족이여','만족이요')
  text = text.replace('사용했니다.','사용했습니다')
  text = text.replace('충전빨리되구죻아요','충전 빨리되고 좋아요')
  text = text.replace('잘됩니다잉','잘됩니다')
  text = text.replace('사용했답니다','사용했어요')
  text = text.replace('가타요','같아요')
  text = text.replace('최곱니다','최고입니다')
  text = text.replace('빠르네요','빠릅니다')
  text = text.replace('기릉','기를')
  text = text.replace('받았습니아','받았습니다')
  text = text.replace('ok.','ok')
  text = text.replace('가져감','가져갑니다')
  text = text.replace('익니다','입니다')
  text = text.replace('수용되오ㅡ','수용되요')
  text = text.replace('맨죡','만족')
  text = text.replace('ㄹ','')
  text = text.replace('구매예정이에요','구매 예정이예요')
  text = text.replace('좋은듓','좋은듯')
  text = text.replace('매우저롐하게잘구매했어요감사하네여','매우 저렴하게 잘 구매했어요 감사해요')
  text = text.replace('만적합니다','만족합니다')
  text = text.replace('ㅈ번창하세요','번창하세요')
  text = text.replace('전이에요','전이예요')
  text = text.replace('조어요','좋아요')
  text = text.replace('젛아요','좋아요')
  text = text.replace('됩니더','됩니다')
  text = text.replace('굿입디다','굿')
  text = text.replace('갘아어요','')
  text = text.replace('ㄷ,','')
  text = text.replace('잘되겠쥬','잘되겠죠')
  text = text.replace('쓰게습니도','쓰겠습니다')
  text = text.replace('하고요','하고')
  text = text.replace('저뇽가방도있네요','전용가방도 있네요')
  text = text.replace('sjsjsjjsjs','') 
  text = text.replace('좋겠구먼요','좋겠네요')
  text = text.replace('좋아욫','좋아요')
  text = text.replace('반퓸합니다','반품합니다')
  text = text.replace('꿋꿋입니다','굿')
  text = text.replace('잘씅게용','잘쓸게요')
  text = text.replace('땡큐네요','감사합니다')
  text = text.replace('베릐굿','굿')
  text = text.replace('효오','')
  text = text.replace('☆','')
  text = text.replace('ᄒ','')
  text = text.replace('썼어ᄉ','썼어요')
  text = text.replace('사용시뉴용할듯욕','사용시 유용할 듯')
  text = text.replace('있아요','있어요')
  text = text.replace('아다아다아','')
  text = text.replace('아다 아다 아','')
  text = text.replace('감사합니다아','감사합니다')
  text = text.replace('goood','굿')
  text = text.replace('싱픔굿배소오쟈웁','상품 굿 배송')
  text = text.replace('쓸겡','쓸게요')
  text = text.replace('굿국국숙','굿굿굿')
  text = text.replace('졸아요','좋아요')
  text = text.replace('USC','USB')
  text = text.replace('A S','AS')
  text = text.replace(' ㅐ외','해외')
  text = text.replace('ᄆ','')
  text = text.replace('ᅳᄋ ᄋ','')
  text = text.replace('배슝','배송')
  text = text.replace('ᅲ','')
  text = text.replace('쓸ᄁ너ᅵᄋ르','쓸게요')
  text = text.replace(' ♥','')
  text = text.replace('ᄏ','')
  text = text.replace('강 추','강추')
  text = text.replace('★','')
  text = text.replace('■','')
  
  # text = text.replace('USB', '유에스비')
  text = text.replace('보관 백', '보관백')
  text = text.replace('TYPE', '타입')
  text = text.replace('C TO C', 'C타입')
  text = text.replace('CTOC', 'C타입')
  text = text.replace('C2C', 'C타입')
  text = text.replace('USB-C', 'USB포트 C타입')
  text = text.replace('USB-A', 'USB포트 A타입')
  text = text.replace('C TYPE', 'C타입')
  text = text.replace('C 타입','C타입')
  text = text.replace('씨타입','C타입')
  text = text.replace('씨 타입', 'C타입')
  text = text.replace('A 타입','A타입')
  text = text.replace('에이타입','A타입')
  text = text.replace('에이 타입', 'A타입')
  text = text.replace('PD 충전','PD충전')
  text = text.replace('PD 잭', 'PD잭')
  text = text.replace('급 속','급속')
  text = text.replace('포드','포트')
  text = text.replace('포드만','포트')
  text = text.replace('포드리','포트')
  text = text.replace('핸폰','핸드폰')
  text = text.replace('휴대폰', '핸드폰')
  text = text.replace('휴대전화', '핸드폰')
  text = text.replace('11볼트', '110V')
  text = text.replace('22볼트', '220V')
  text = text.replace('11V', '110V')
  text = text.replace('22V', '220V')
  # text = text.replace('35W', '35와트')
  # text = text.replace('45W', '45와트')
  # text = text.replace('65W', '65와트')
  text = text.replace('포터', '포트')
  # text = text.replace('USB', 'USB포트') 
  text = text.replace('USB 포트', 'USB포트')
  text = text.replace('동시 충전', '동시충전')
  text = text.replace('고속 충전', '고속충전')
  text = text.replace('동시 이용', '동시이용')
  text = text.replace('동시 사용', '동시사용')
  text = text.replace('여러 나라', '여러나라')
  text = text.replace('코넥터', '커넥터')
  # 1개짜리는 제외

  #### pad 데이터 추가
  text = text.replace('페드','패드')
  text = text.replace('스마트 스토어', '스마트스토어')
  text = text.replace('구매가입니다','구매 각입니다')
  text = text.replace('강쥐','강아지')
  text = text.replace('웨리코기','웰시콜리')
  text = text.replace('정 사이즈','정사이즈')
  text = text.replace('습수 력','흡수력')
  text = text.replace('가겍대비','가격 대비')
  text = text.replace('구먀','구매')
  text = text.replace('재재 재주 뭉','재주문')
  text = text.replace('핫 디','할인')
  text = text.replace('여기 거만','여기 것만')
  text = text.replace('제 제제 그메','재구매')
  text = text.replace('저령 렴해서','저렴해서')
  text = text.replace('배변 훈련','배변훈련')
  text = text.replace('요건','이것')
  text = text.replace('몰티즈','말티즈')
  text = text.replace('쟁여두고 습니다','쟁여두고 씁니다')
  text = text.replace('흡수 잔','흡수 잘')
  text = text.replace('만족함 니아','만족합니다')
  text = text.replace('8587그램',' 85G 87G')
  text = text.replace('배 편견','배변견')
  text = text.replace('노령견','노견')
  text = text.replace('8킬로','8KG')
  text = text.replace('개님','개가')
  text = text.replace('편핮니당','편합니다')
  text = text.replace('킬로','KG')
  text = text.replace('ㅏ용','사용')
  text = text.replace('대형 패드','대형패드')
  text = text.replace('배편 판','배변판')
  text = text.replace('R가 성비짱입니다','가성비 짱입니다')
  text = text.replace('파견 가정','파견가정')
  text = text.replace('위생저이에여','위생적이예요')
  text = text.replace('그램','G')
  text = text.replace('가나다라마바사아자차카타파하','')
  text = text.replace('가실 비','가성비')
  text = text.replace('댕댕이','강아지')
  text = text.replace('망상 이 패들만 써요','항상 이 패드만 써요')
  text = text.replace('같은야ㅂ은50','같은 양')
  text = text.replace('쉬야','쉬')
  text = text.replace('유기견 보호소','유기견보호소')
  text = text.replace('갱에 지','강아지')
  text = text.replace('잘약형여 기껏만 습니다','절약형 여기 것만 씁니다 ')
  text = text.replace('매 번지ㅢ구매하는','매번 재구매하는')
  text = text.replace('거성비','가성비')
  text = text.replace('가성 배짱입니다','가성비 짱입니다')
  text = text.replace('싸능데','싸는데')
  text = text.replace('또 주 무 할게요','또 주문할게요')
  text = text.replace('재구 매각','재구매 각')
  text = text.replace('가성 비각','가성비 각')
  text = text.replace('VERY GOOD PRODUCT','아주 좋은 제품')
  text = text.replace('N 번째','N번째')
  text = text.replace('피숑','비숑')
  text = text.replace('논 슬립','논슬립')
  text = text.replace('계소','계속')
  text = text.replace('떨어 지명','떨어지면')
  text = text.replace('대대 만족','대만족')
  text = text.replace('재ㅣ매','재구매')
  text = text.replace('국구닫ㄱ다다다가구구숫ㄱ국','')
  text = text.replace('강지','강아지')
  text = text.replace('성 비','가성비')
  text = text.replace('더드러가여','더 들어가요')
  text = text.replace('배변 판','배변판')
  text = text.replace('퍼피','강아지')
  text = text.replace('가성이','가성비')
  text = text.replace('20G38G40G','20G 38G 40G')
  text = text.replace('이 패들만','이 패드만')
  text = text.replace('성비','가성비')
  text = text.replace('아가','아기')

  # wireless 추가
  text = text.replace('먼지 통제 거할 때','먼지통 제거 할 때')
  text = text.replace('흡입 능력','흡입능력')
  text = text.replace('시스탬','시스템')
  text = text.replace('선책','선택')
  text = text.replace('사셈','사세요')
  text = text.replace('워터 루핀','워터루핀')
  text = text.replace('상풍','상품')
  text = text.replace('청소 기밀','청소기 밀')
  text = text.replace('청소 기밀기','청소기 밀기')
  text = text.replace('진 작살','진작 살 ')
  text = text.replace('먼지 통안','먼지통 안')
  text = text.replace('먼 지통','먼지통')
  text = text.replace('먼지 통','먼지통')
  text = text.replace('해르','헤드')
  text = text.replace('가법고','가볍고')
  text = text.replace('삼실','사무실')
  text = text.replace('화이트로','화이트')
  text = text.replace('마니','많이')
  text = text.replace('기욤','귀여움')
  text = text.replace('소제','소재')
  text = text.replace('아소변워요','아쉬워요')
  text = text.replace('아소변우나','아쉬우나')
  text = text.replace('보 풀제 거기','보풀제거기')
  text = text.replace('강화 마루','강화마루')
  text = text.replace('청소 기안에','청소기 안에')
  text = text.replace('흠이 버력','흡입력')
  text = text.replace('누름','누르는')
  text = text.replace('청소역','청소력')
  text = text.replace('채상','책상')
  text = text.replace('청소용임','청소용')
  text = text.replace('가 벼 워서','가벼워서')
  text = text.replace('착지만','작지만')
  text = text.replace('강학요','강해요')
  text = text.replace('머리 타락','머리카락')
  text = text.replace('소올찍히','솔직히')
  text = text.replace('가분','기분')
  text = text.replace('착 아도','작아도')
  text = text.replace('늪이','높이')
  text = text.replace('헤어 두레기','헤어드라이기')
  text = text.replace('앙고','않고')
  text = text.replace('자사소형핸ㄷㄱ청소기','자사 소형 핸드청소기')
  text = text.replace('흡립','흡입')
  text = text.replace('구래도','그래도')
  text = text.replace('정소','청소')
  text = text.replace('최인','최신')
  text = text.replace('작은걸찾았늣데','작은 걸 찾았는데')
  text = text.replace('조그 마한','조그마한')
  text = text.replace('과대광 고언 것 같아요','과대광고 인 것 같아요')
  text = text.replace('반려 묘도','반려묘도')
  text = text.replace('탁입니다','딱입니다')
  text = text.replace('필라 센터','필라테스 센터')
  text = text.replace('주목적','목적')
  text = text.replace('총후 기','청소기')
  text = text.replace('좋진요','좋아요')
  text = text.replace('한보 따립니다','한보따리 입니다')
  text = text.replace('공력','진공력')
  text = text.replace('탈 거 착임','탈거착이')
  text = text.replace('문특','문틀')
  text = text.replace('어주','아주')
  text = text.replace('스투디오','스튜디오')
  text = text.replace('색원','색감')
  text = text.replace('행보해요','행복해요')
  text = text.replace('뒤 덮인','뒤덮인')
  text = text.replace('뎔심','결심')
  text = text.replace('순각','순간')
  text = text.replace('공홈','공식 홈페이지')
  text = text.replace('배턴','배터리')
  text = text.replace('ㅊ','후 차')
  text = text.replace('대파 이프','대 파이프')
  text = text.replace('위봉','위 봉')
  text = text.replace('신준','사준')
  text = text.replace('구서 무석','구석구석')
  text = text.replace('활둉도면','활용도 면')
  text = text.replace('삼성청노기','삼성 청소기')
  text = text.replace('역조','욕조')
  text = text.replace('완저','완전')
  text = text.replace('망어','망이')
  text = text.replace('오자마다','오자마자')
  text = text.replace('안족스럽다네요','만족스럽다네요')
  text = text.replace('장봉익','장봉이')
  text = text.replace('아전','아주')
  text = text.replace('상각','생각')
  text = text.replace('선무','선물')
  text = text.replace('만지','먼지')
  text = text.replace('원하셩거든요','원하셨거든요')
  text = text.replace('싼 마이지만','싼마이지만')
  text = text.replace('긴가 민가','긴가민가')
  text = text.replace('알지게','알차게')
  text = text.replace('흡입려콰','흡입력과 ')
  text = text.replace('자몽할 듯','사용할 듯')
  text = text.replace('정마','정말')
  text = text.replace('가성비 절고','가성비 좋고')
  text = text.replace('뽀송','보송')
  text = text.replace('아음','마음')
  text = text.replace('엄니','어머니')
  text = text.replace('기대 이싱','기대 이상')
  text = text.replace('난립','날림')
  text = text.replace('먼지 날림','먼지날림')
  text = text.replace('파어','파워')
  text = text.replace('소응','소음')
  text = text.replace('가법고','가볍고')
  text = text.replace('너무 귀영 와요','너무 귀여워요')
  text = text.replace('아무래고','아무래도')
  text = text.replace('추전','추천')
  text = text.replace('오시프텔','오피스텔')
  text = text.replace('청소 용구','청소용품')
  text = text.replace('핵 좋음','정말 좋음')
  text = text.replace('엘사','LG')
  text = text.replace('봏았습니다','좋았습니다')
  text = text.replace('청소가 소변워요','청소가 쉬웠어요')
  text = text.replace('총 소포','청사포')
  text = text.replace('탑 오브 탑','탑오브탑')
  text = text.replace('마논','만원')
  text = text.replace('천소','청소')
  text = text.replace('훵씬','훨씬')
  text = text.replace('소변워요','쉬웠어요')
  text = text.replace('스턴딩','스탠딩')
  text = text.replace('그으긋','굿')
  text = text.replace('흡인력','흡입력')

  #### 레깅스
  text = text.replace('Y 라인','Y존')
  text = text.replace('아이존','Y존')
  text = text.replace('와이 존','Y존')
  text = text.replace('와이존','Y존')
  text = text.replace('젝 스미스','잭시믹스')
  text = text.replace('Y 존','Y존')
  text = text.replace('네 링스','레깅스')
  text = text.replace('좋습니 디좋습니 디좋습니 디','')
  text = text.replace('2XL','XXL')
  text = text.replace('3XL', 'XXXL')
  text = text.replace('엑스스몰', 'XS')
  text = text.replace('에스', 'S')
  text = text.replace('미디엄', 'M')
  text = text.replace('라지', 'L')
  text = text.replace('엑스라지', 'XL')
  text = text.replace('투엑스라지', 'XXL')
  text = text.replace('엑스 라지', 'XL')
  text = text.replace('투엑스 라지', 'XXL')
  text = text.replace('쫄바지','레깅스')
  text = text.replace('아시게도','아쉽게도')
  text = text.replace('엉덩일','엉덩이를')
  text = text.replace('44 사이즈','44사이즈')
  text = text.replace('55 사이즈','55사이즈')
  text = text.replace('66 사이즈','66사이즈')
  text = text.replace('77 사이즈','77사이즈')
  text = text.replace('88 사이즈','88사이즈')
  text = text.replace('44','44사이즈')
  text = text.replace('55','55사이즈')
  text = text.replace('66','66사이즈')
  text = text.replace('6677','66사이즈 77사이즈')
  text = text.replace('면인','면이')
  text = text.replace('바이커 쇼','바이커쇼츠')
  text = text.replace('홈트', '홈트레이닝')
  text = text.replace('히프 업', '힙업')
  text = text.replace('히프', '힙')
  text = text.replace('아소변움', '아쉬움')
  text = text.replace('아소변운', '아쉬운')
  text = text.replace('구슥슥슥슥슥슥슥슥슥', '')
  text = text.replace('하비', '하체비만')
  text = text.replace('FREE 사이즈', '프리사이즈')
  text = text.replace('읎음', '없음')
  text = text.replace('고퀄', '고퀄리티')
  text = text.replace('필라', '필라테스')

  #### 마스크팩
  text = text.replace('메디 힐먼','메디힐먼')
  text = text.replace('최구','최고')
  text = text.replace('마디 힐','메디힐')
  text = text.replace('똥 손','똥손')
  text = text.replace('더마 픽스','더마픽스')
  text = text.replace('앰풀','앰플')
  text = text.replace('리들 샷','리들샷')
  text = text.replace('머디 힐','메디힐')
  text = text.replace('충실다','충실')
  text = text.replace('화큰거림','화끈거림')
  text = text.replace('따끔 거림','따끔거림')
  text = text.replace('엄첟','엄청')
  text = text.replace('호과','효과')
  text = text.replace('잔느 식스 헬로 케어 팩','잔느식스헬로케어팩')
  text = text.replace('쟁여 노코','쟁여놓고')
  text = text.replace('뒤끔 찌','뒤끔치')
  text = text.replace('미 백수 분','미백 수분')
  text = text.replace('베슷흐','베스트')
  text = text.replace('넉 넛','넉넉')
  text = text.replace('넛어두','넣어두고')
  text = text.replace('메딜힐','메디힐')
  text = text.replace('지대로','제대로')
  text = text.replace('최애 텀','최애탬')
  text = text.replace('깜ㄴ장색','검정색')
  text = text.replace('히알루론','히알루로닉')
  text = text.replace('효고ㅏ','효과')
  text = text.replace('효고','효과')
  text = text.replace('앰프','앰플')
  text = text.replace('아이 패치','아이패치')
  text = text.replace('붇','붙')
  text = text.replace('피분','피부')
  text = text.replace('순수 크린','순수크린')
  text = text.replace('흘러 내림','흘러내림')
  text = text.replace('닥터 웰메이드','닥터웰메이드')
  text = text.replace('서우 콜라겐','서우콜라겐')
  text = text.replace('피르 이즘','피르이즘')
  text = text.replace('피르 가즘','피르이즘')
  text = text.replace('일 인일 팩','일인일팩')
  text = text.replace('진덕','진득')
  text = text.replace('스마트 스토어','스마트스토어')
  text = text.replace('골드 핏 마스크','골드핏마스크')
  text = text.replace('매디 힐리','메디힐')
  text = text.replace('좋아 거 같아요',' 좋은 것 같아요')
  text = text.replace('1일 1팩','일인일팩')
  text = text.replace('패기','팩이')
  text = text.replace('그린 토마토','그린토마토')
  text = text.replace('핑크 콜라겐','핑크콜라겐')
  text = text.replace('피부사','피부가')
  text = text.replace('훠어얼씬','훨씬')
  text = text.replace('GGGOOODDD','굿')
  text = text.replace('개래해패패메이딘에리이데니미에이프','')
  text = text.replace('적당하니갠출하네요','적당하니 괜찮네요')
  text = text.replace('생각의 에로','생각 이외로')
  text = text.replace('조흥','좋음')
  text = text.replace('다조 음','다 좋음')
  text = text.replace('트러볼리','트러블이')
  text = text.replace('조하','좋아')
  text = text.replace('잘 신거','잘 산것')
  text = text.replace('미 백면에','미백 면에')
  text = text.replace('해시네요','하시네요')
  text = text.replace('퍀','팩')
  text = text.replace('고 머','구매')
  text = text.replace('배송잉','배송이')
  text = text.replace('포정','포장')
  text = text.replace('써봏게요','써볼게요')
  text = text.replace('로로 스키니','로로스키니')
  text = text.replace('처촉쓰','촉촉')
  text = text.replace('마스크퍅','마스크팩')
  text = text.replace('두문','주문')
  text = text.replace('다 회차','다회차')
  text = text.replace('내 용품엔','내용품에는')
  text = text.replace('누런색','노란색')
  text = text.replace('만복','만족')
  text = text.replace('후니아거저리디저리이자라라','')
  text = text.replace('에셍스','에센스')
  text = text.replace('도아오','좋아요')
  text = text.replace('마크 스펙','마스크팩')
  text = text.replace('촉촉촉 촉촉해요','촉촉해요')
  text = text.replace('촉촉촉 너무 촉촉해요','촉촉해요')
  text = text.replace('대 요량','대용량')
  text = text.replace('뒤꿈 찌','뒤꿈치')
  text = text.replace('좀따끔하지먀미백됩니다','좀 따끔하지만 미백됩니다')
  text = text.replace('무착','무척')
  text = text.replace('업고','없고')
  text = text.replace('달팽이 팩','달팽이팩')
  text = text.replace('두름','주름')
  text = text.replace('즛 보여요','듯 보여요')
  text = text.replace('시웡해서','시원해서')
  text = text.replace('허낙ㄱㄴ거내거니러니니','')
  text = text.replace('재서 매는','재구매는')
  text = text.replace('악전 조라','악건조라')
  text = text.replace('조은진','좋은지')

  return text

In [ ]:
df['add_rule_based_split_review'] = df['add_rule_based_split_review'].apply(replace_text)

In [ ]:
def clean_text(text):
    if isinstance(text, str):
        return re.sub(r'[-=+,#/\?:^$.@*"※~&%ㆍ!』\\‘·|\(\)\[\]\<\>`\'…\"\“’]', '', text)
    else:
        return ""

In [ ]:
product['add_rule_based_split_review'] = product['add_rule_based_split_review'].apply(clean_text)

In [ ]:
from konlpy.tag import Mecab
m = Mecab(dicpath = 'C:/mecab/mecab-ko-dic')

In [ ]:
def preprocess_reviews(reviews, stopwords_path, additional_stopwords=[]):
    # 불용어 파일 로드
    with open(stopwords_path, encoding='utf-8') as file:
        stopwords = file.read().splitlines()
    
    # 추가 불용어 합치기
    stopwords.extend(additional_stopwords)

    # 리뷰 전처리
    clean_sent = []
    for review in reviews:
        clean = re.sub(r'[-=+,#/\?:^$.@*"※~&%ㆍ!』\\‘·|\(\)\[\]\<\>`\'…\"\“’]', '', review)
        clean = clean.replace('\n', '').replace('\r', '')  # 줄바꿈 문자 제거
        clean_sent.append(clean)

    m = Mecab(dicpath = 'C:/mecab/mecab-ko-dic')
    
    word_list = []
    for sentences in clean_sent:
        # 정규화
        # sentences = m.normalize(sentences)
        # 토큰화
        # tokens = sentences.split()
        tokens = m.nouns(sentences)
        # tokens = okt.nouns(sentences)
        # 불용어 제거
        tokens = [token for token in tokens if token not in stopwords]
        word_list.append(tokens)
    
    return word_list

In [ ]:
"""
stopwords_path = "C:/Users/songy/Desktop/Data/stopwords-ko.txt"
with open(stopwords_path, encoding='utf-8') as file:
    stopwords = file.read().splitlines()

stopwords.extend(additional_stopwords)
print(stopwords)
"""

In [ ]:
def compiled_morphs(texts, stopwords_path, additional_stopwords=[]):
    # 불용어 파일 로드
    with open(stopwords_path, encoding='utf-8') as file:
        stopwords = file.read().splitlines()

    # 추가 불용어 합치기
    stopwords.extend(additional_stopwords)

    # 리뷰 전처리
    clean = re.sub(r'[-=+,#/\?:^$.@*"※~&%ㆍ!』\\‘·|\(\)\[\]\<\>`\'…\"\“’]', '', texts)
    clean = clean.replace('\n','').replace('\r','')     # 줄바꿈 문자 제거

    # 형태소 분석기 추가
    m = Mecab(dicpath='C:/mecab/mecab-ko-dic')

    result = []
    for i in range(len(m.pos(clean))):
        word, pos = m.pos(clean)[i]
        # 형태소가 연결 어미인지 확인
        if pos in ['VA']:
            if i + 1 < len(m.pos(clean)):
                next_word, next_pos = m.pos(clean)[i + 1]
                if next_pos in ['EC', 'ETM']:           # 연결 어미 또는 관형형 전성 어미
                    result.append(word + next_word)
                else:
                    result.append(word + next_word)
        elif pos in ['NNG', 'NNP']:
            result.append(word) 
    
    # 불용어 처리
    results = [word for word in result if word not in stopwords]
    
    return results

In [ ]:
def compiled_morphs_noun(texts, stopwords_path, additional_stopwords=[]):
    # 불용어 파일 로드
    with open(stopwords_path, encoding='utf-8') as file:
        stopwords = file.read().splitlines()

    # 추가 불용어 합치기
    stopwords.extend(additional_stopwords)

    # 리뷰 전처리
    clean = re.sub(r'[-=+,#/\?:^$.@*"※~&%ㆍ!』\\‘·|\(\)\[\]\<\>`\'…\"\“’]', '', texts)
    clean = clean.replace('\n','').replace('\r','')     # 줄바꿈 문자 제거

    # 형태소 분석기 추가
    m = Mecab(dicpath='C:/mecab/mecab-ko-dic')

    result = []
    for i in range(len(m.pos(clean))):
        word, pos = m.pos(clean)[i]
        if pos in ['NNG', 'NNP']:
            result.append(word) 
    
    # 불용어 처리
    results = [word for word in result if word not in stopwords]
    
    return results

In [ ]:
example_sent = "퓨즈 교체가 자동이라서 신경 쓸 일이 하나 줄은 것도 장점이고요"

In [ ]:
def find_morphs(text):
    """
    명사, 형태소 추출 함수 (형용사의 연결 어미를 반영해 추출)
    """
    result = []

    for i in range(len(m.pos(text))):
        word, pos = m.pos(text)[i]
        # 형태소가 연결 어미인지 확인
        if pos in ['VA']:
            if i + 1 < len(m.pos(text)):
                next_word, next_pos = m.pos(text)[i + 1]
                if next_pos in ['EC', 'ETM']:           # 연결 어미 또는 관형형 전성 어미
                    result.append(word + next_word)
                else:
                    #result.append(word + next_word)
                    result.append(word)
        elif pos in ['NNG', 'NNP']:                     # 명사 
            result.append(word)

    return result

In [ ]:
find_morphs("튼튼한 것 견고한 컴팩트 케이스 테스트")

In [ ]:
compiled_morphs("리뉴얼", stopwords_path="C:/Users/songy/Desktop/Data/stopwords-ko.txt")

In [ ]:
compiled_morphs_noun("리뉴얼", stopwords_path="C:/Users/songy/Desktop/Data/stopwords-ko.txt")

In [ ]:
#compiled_morphs("", stopwords_path="C:/Users/songy/Desktop/Data/stopwords-ko.txt")

In [ ]:
#product['changed_text'] = product['add_rule_based_split_review'].apply(find_morphs)
reviews = product['add_rule_based_split_review']  # 리뷰 데이터
stopwords_path = "C:/Users/songy/Desktop/Data/stopwords-ko.txt"

additional_stopwords = ['가', '가격', '가운데', '가족', '간', '감', '감비', '감사', '값', '강추', '같고', '같습니다', '같아', '같아서', '같아요', '겉'
                        '같은', '결제', '같은데', '개', '거', '거랑', '걱정', '검색', '것', '겉', '게', '게요', '결과', '결재', '경우', '고', '금액', '차이',
                        '고려', '고만', '고민', '곳', '과', '괜찮아요', '구리고', '구매', '구매평', '구비', '구성', '구입', '군더더기', '굿', '귀찮게',
                        '귀찮고', '글', '급하게', '기대', '기존', '기준', '길', '김기', '깊질', '까지', '끝', '나라', '나중', '날', '남편',
                        '냐짱', '너무', '네이버', '노브', '눈', '느낌', '느리네요', '는', '다', '다르게', '다르고', '다양', '다이소', '다행', '달',
                        '담', '답변', '대부분', '대비', '대신', '덕분', '덜렁', '도', '도착', '돈', '동봉', '동생', '동일', '돼다', '되다',
                        '되어다', '두면', '뒤', '들', '등', '디', '딸', '때', '땡', '뛰어나다', '라', '라고', '래빗', '레이', '로', 
                        '를', '리뷰', '마', '마련', '마음', '만', '만족', '많고', '많아서', '많았었', '많지만', '말', '맘', '맛', '멀티',
                        '멜리', '면', '못', '무료', '문', '문제', '물', '물건', '미', '및', '밑', '며칠', '바이', '박스', '반', '받다',
                        '발', '발송', '배', '배송', '번', '번개', '벽', '변', '별도', '별점', '보관', '복', '봉', '부분', '브레',
                        '비선', '비싸지', '비싸지만', '빌', '사나', '사서', '사실', '사요', '사용', '사장', '사진', '생각', '서', '선물', '선택',
                        '설치', '세상', '쇼퍼', '숍', '수', '숙소', '스타', '스티커', '스펙', '시', '시기', '시대', '시험', '식겁', '신랑',
                        '신행', '실사', '실사용자', '싸게', '싸구려', '싸는데', '싸진', '쌀', '써다', '쏙', '쓸모', '아들', '아무것', '아쉽네요', '아쉽습니다',
                        '아주', '아직', '안', '않다', '알차게', '앞', '애', '야지', '얇아요', '얘기', '어렵더라고요', '어머니', '언니', '언스', '엄마',
                        '업', '없겠', '없고', '없나', '없네요', '없는', '없어서', '없어요', '없었', '없으니까요', '없지만', '에', '에서', '엔', '엔코',
                        '여분', '여유', '여행', '역할', '연락', '연휴', '옆', '예비', '예전', '옛날', '오다', '오랫동안', '오전', '오케이', '오후',
                        '와', '와이프', '요', '요새', '요즘', '용', '운', '워홀', '원', '원래', '유용', '으로', '은', '을', '의',
                        '이', '이거', '이다', '이라', '이랑', '이번', '이외', '이용', '이유', '이전', '인', '인터넷', '일행', '있고', '있군요',
                        '있긴', '있네요', '있는', '있는대도', '있다', '있다는', '있던데', '있어', '있어서', '있었', '있을', '있지만', '자', '자다', '자체',
                        '잔', '잘', '장만', '잦은데', '저렴', '저번', '전', '전용', '전이', '전체', '점', '정', '정도', '제', '제공',
                        '제품', '종결', '종류', '좋겠', '좋고', '좋네요', '좋더라구요', '좋습니다', '좋아서', '좋아요', '좋았', '좋은', '좋은데요', '좋을', '주', '좋',
                        '주니', '주문', '주파', '준비', '중요', '지난번', '지네', '지원', '지인', '집', '차', '차지', '처리', '처음', '첨부',
                        '추천', '추후', '출국', '출장', '친구', '친절', '커피', '코', '크게', '크지', '큰일', '테스트', '퇴원', '튀', '트',
                        '티', '판매', '편리', '편하게', '편하고', '평', '평생', '포장', '포함', '표', '프리', '필', '필수탬', '필요', '하고', '필수', '이거', '아쉽더라구요', 
                        '하다', '하머', '하우스', '한', '한국', '한큐', '해', '형', '호텔', '확인', '활용', '회사', '효', '후', '후기', '같', '없', '당황', '목', '탬', '높은',
                        '흠', '히', '힘', '하루','단','역시','음','지고','이기','이제','무거워서','주려','하나라','가능','가성비','배변','패드','견','강아지',
                        '해외','여행','어댑터',
                        
                        # pad stopword 추가
                        '성비', '짱', '위', '질', '이것저것', '막', '재재', '아조', '보통', '평소', '이후', '정기', '울', '량', '대박', '단골', '계속', '포', '여기저기', '조아', 
                        '이틀', '지금', '후회', '뒷면', '참고', '푸', '첨', '다둥', '갑', '패', '한동안', '완전', 'ㄴ', '조', '숯', '킹스',
                        '멍멍이', '향도', '가성', '앞뒤', '사이', '방', '코코', '이름', '최근', '딘', 'ㄱ', '짜리', '비', '급', '드', '중', '한쪽', '쥐', '작년', '룸', '송', '사면',
                        '견이', '남자', '불가', '갈기', '말티', '제일', '속', '솜', '별', '미만', '고요', '요귀', 'ㄷ', '선비', '수컷', '초대', 'ㅣ', '쏘', '럭', '오늘', '오랜만', 
                        '아지', '무리', '낮', '인생', '기', '껍지', '최소', '중대', '옹', '여자', '탓', 'ㅂ', '귀퉁이', '스피츠', '덕', '조타', '국민', '남', '톱', '품', '독', 
                        '왼쪽', '오른쪽', '강', '만점', '진작', '내용', '깔', '플로', '근처', '임', '초코', '조음', '세로', '성', '사도', '묘', '댁', '미세', '쇼핑', '전화', '탑', 
                        '답', '상세', '빅', '후원', '바깥', '프렌치', '덩', '여아', '치', '지', '대', '쟁', '펠', '렛', '갱', '제격', '역', '둥이', '절', '실', '습수', '멈', '겨', 
                        '가설비', '족', '적', '신문지', '회용', '믹', '페스', '워', '이대로', '반년', '응아', '밧', '현상', '방법', '홈', '숩', '반복', '도통', '이코노미', '새요', 
                        '셉', '인향', '중자', '최', '수력', '액', '시중', '뒤처리', '엉망', '바도', '콩', '짐', '장당', 'ㅁ', '평균', '이동', '구요', '말모', '사니', '여러', '유', 
                        '중임', '아진', '장모', '매합', '완료', '한참', '해당', '감안', '명', '조심', '역대', '찜', '믈', '통', '몽이', '향상', '샙', '꼬미', '밤중', '최곤', '포미',
                        '뎅이', '여요', '고생', '문자', '반나절', '최상', '대용', '녀', '요조', '아요', '반경', '팬', '쉬만', '시도', '렴', '생가', '감도', '우', '자요', '암', '후론', 
                        '루루', '은근', '좃', '띠', '저령', '뉴', '항사', '귀양', '붐', '펄프', '노모', '이쪽저쪽', '세장', '뮨', '규', '꼴', '파드', '니노', '괴안', '슈', '우타다', 
                        '주세', '시야', '희', '뚱', '씨', '잴', '실례', '앒', '와중', '끝판', '추', '견지', '요배', '낫', '동', '에다', '조앙', '스탠', '세지', '연', '니더', '바', '닽', 
                        '양이', '글쎄', '사본', '단독', '쓰봉', '친정집', '김', '치와', '슈거', '느', '토이', '특이', '방치', '군', '욕', '루피', '지에', '흘', '부', '견응', '살짝', '개아들', 
                        '텨', '듄', '이하', '앝', '아용', '주말', '턱', '다뇨', '알', '상', '넝', '쪼끔', '야기', '직', '붕가', '쥬', '외', '서오', '아디', '로뎀', '젖', '덧', '현재', '번창', '작', '항', '불', '적정', 
                        '안세', '소자', '시판', '끝단', '어림', '냄', '서요', '위주', '깔개', '업장', '내외', '울대', '주위', '찰스', '버드맨', '칭', '베리', '푹신', '삿', '대응', '논의', '경', '국', 
                        '소취', '산', '밤새', '마린', '임도', '개모', '디즈', '팝', '슈퍼', '립니', '특유', '다회', '곳곳', '파', '대소', '쏭', '순식간', '줌', '깜놀', '진심', '동공', '치곤', '후손', 
                        '로랑', '형성', '토모', '땅', '약형', '동군', '여기사', '뚜', '룡', '조그', '일쑤', '강주', '척', '겐', '뚜꾸', '하마', '둔갑', '수매', '겡', '해영', '러버', '대로', '습기', '쉽', 
                        '루', '고중', '긑', '우니', '낙', '넉', '기니', '화요일', '개딸', '재결', '날강도', '목요일', '월요일', '땜질', '야간', '이해', '판위', '장여', '커크', '랜', '도라', '재근', '호도', 
                        '샌', '파파', '외각', '쿠거', '몽중', '양도', '개놈', '가이', '대피', '장실', '고령', '일곡', '감이', '만자문', '적격', '길거리', '덩이', '선방', '키런', '햄', '께랑', '개론', '램', '와이오밍', 
                        '완존', '타비', '찰', '잡잡', '요아', '나오', '어감', '홍시', '두마', '감사용', '산이', '키우', '쥐똥', '달견', '부들', '부실', '징', '둡', '밀리지', '구람', '어치', '엉망진창', '큰딸', '초아', 
                        '텐', '저당', '쉼', '휠', '견반', '거려', '제로', '예견', '역쉬', '항인', '용기', '별전', '유어', '듀야', '유야', '유라', '오야', '엉', '쟁기', '밸', '용허', '죠', '누질', '잇', '정에', '간혼', 
                        '조조', '오조', '지정', '장호', '굽', '보타', '우고', '보꾹', '시키', '수놈', '곁', '양정', '불량아', '소홀', '우거', '꽁', '숫', '닼', '난사', '욤', '수개월', '롱이', '성경', '임료', '요구', 
                        '줄장', '쓰래', '대몽', '잭', '사항', '거유', '계족', '둠', '과찬', '아서', '편이', '벌', '당사', '개더', '애요', '번차', '납', '라주', '솧', '일회', '테', '바야', '엠', '다도', '요개', '봄', '가을', 
                        '애장', '고고씽', '버', '비로', '대기', '그럭', '땡땡이', '강선', '스키', '버논', '비가', '거짓', '대요', '호오', '저초', '야호', '비축', '진전', '진리', '긴말', '미고', '두아', '타우', '왕따', '꺼만', 
                        '두기', '항거', '짤짤이', '삽', '돌아악', '영역', '보름', '도중', '私', '房', '菜', '美', '아즈', '셀리', '옌', '버리', '휴지통', '매년', '안방', '위협', '쓰개', '깡깡이', '미미', '이리리', '비비', 
                        '디디', '복불복', '션', '노드', '몽', '홀', '점입', '콜', '자우', '어래', '누나야', '룬', '어류', '차선', '토리', '금대', '리치', '니즈', '불안전', '절지', '개로신', '미안', '레트', '수별', '번갈', 
                        '드로', '가스', '가송', '비갑', '유니참', '계', '안성', '긔', '애고', '강순', '시예', '시위', '중화', '문가', '거어', '려', '함', '익만', '승수', '두품', '거만', '극', '시쟁', '기격구', '담날', 
                        '유랑', '가계', '거보', '먹', '우씨', '시바', '요일', '중닭', '시골집', '방생', '모카', '거미', '떨기', '해유', '뱃', '샅', '아누', '아영', '턴', '몰리', '스', '농수', '산집', '이란', '세보', '하소연', '제구', 
                        '보대', '세뇨', '도꾸', '소모량', '빈', '예시', '동급', '참조', '변환', '졸리', '래브라도', '싱', '물가고', '녕', '가드', '짝', '사여', '소성', '거도', '사유', '상기', '지라', '담부', '대건', '미흡', '샵', 
                        '저주', '폭탄', '살', '용굿', '검', '저어', '않', '이상일', '깜장', '피', '광증', '조코', '안뜸', '마나', '골', '까치', '셔', '애덕', '바이바이', '돗돔', '여타', '휘', '금새', '곡', '잎', '이즈라', '조기', '이길만', 
                        '범', '야옹이', '비론', '톰', '제때', '똘', '도툼', '조야', '촉', '매기', '종전', '가량', '이대', '판티', '갗', '시량', '당량', '시미', '움', '소용', '톡', '용양', '약시', '병', '두깨', '재문', '서두', '피츠', '필도', '방기', 
                        '사수', '불리', '자신감', '뒤편', '쌍', '이재', '막지', '따님', '규조토', '티즈', '보', '농', '계임', '난중', '러그', '밝은색', '로럴', '젤', '하항', '상쓰', '궁', '향이', '아드', '대왕', '소영', '선정', '소지', '종가', 
                        '기원', '오래간만', '특수', '개춘', '기라', '보동', '앞쪽', '깃', '표정', '샆', '재탕', '증발', '정성', '론', '갱지', '네요', '라다', '얄', '골치', '염용', '뜸', '자류', '가디', '견분', '발도', '장꽝', '샤', '비위', '의견', 
                        '리', '주무', '시바이', '액셀', '곡산', '종로', '봉사', '숳', '구예', '요령', '즈', '몰기', '보거', '래', '한물', '에소', '망사', '의촉', '공함', '뉴뉴', '패두', '쥴', '애벌빨래', '애용품', '전거', '반련', '견제', '시일', 
                        '수록', '페트', '츠', '아가야', '재그', '도치', '양견', '서리', '버디', '쿠싱', '맘마', '베베', '페페', '공', '출근길', '잠', '개당', '하주', '개도', '배번', '시퍼', '사세', '변태', '직사각', '사열', '건승', '이력', '교', 
                        '승부', '복고', '입소', '모랑', '바요', '탐', '최선', '광', '외면', '낌', '타가', '문턱', '태무', '무시', '모자람', '문란', '경양', '여럿', '기판', '대서', '광색', '말캉', '아간', '이먼', '개의', '이강', '오공', '귀염둥이', 
                        '지독', '변양', '존망', '그간', '모도', '셀', '소라지', '아버님', '죄', '매달', '민공', '어멈', '부스럼', '최저', '본지', '갑오', '브', '재자', '페트로', '끈끈이', '백패', '그제', '모에', '수문', '마은', '가섭', '큐', 
                        '엇청', '피피', '군요', '본론', '로러', '오노', '이이', '뚱뚱이', '뚝', '몬스터', '곰팡', '와부', '확연', '퍼스트', '세미', '리라', '평일', '모레나', '라이', '력', '벗', '용용', '개재', '북송', '개코', '두던', '하루치', 
                        '볕', '재거', '지난달', '색이', '로지', '줄', '화', '까탈', '저반', '자마', '대오', '리프', '견초', '터짐', '보디', '테일', '겹', '세', '최악', '려고', '버니', '데이', '대다수', '세어', '하번', '교하', '긁', '롬', '반낭', 
                        '범벅', '자죽', '자선', '무조','요요','가가'
                        
                        # wireless 추가
                        '부담', '가요', '웰', '우수', '신일', '버전', '어머님', '핸드', '세요', '리아스', '비스', '중국', '할머니', '샤크', '윤', '릭', '누나', '어버이날', '링', '엘', '텍', '상관', '아이니', '반면', '블로', '셈', '긴박', '귀욤', 
                        '파필', '달째', '뜻', '종', '찰나', '닉', '마이', '화이', '당', '구가', '웰리', '다이', '밀레', '롱', '익', '소', '열', '터', '요란', '상풍', '위나', '한몫', '번호', '통안', '킴', '헤르', '법고', '삼실', '빠', '럼', '겸용',
                        '산지', '관건', '잉', '덤', '양', '가라', '용의', '칸', '제우스', '건', '유요', '기욤', '판', '오래', '최대한', '자봉', '비질', '릴', '적당', '자방', '처남', '보고', '흡력', '고고', '그만큼', '핫', '레', '평보', '통이', '내일', 
                        '유의', '첫날', '보가', '단기간', '자니', '경악', '팡', '방방', '선책', '이내', '킹', '보충', '킬', '고모님', '뻥', '붓', '자바라', '가닥', '발휘', '그중', '개편', '의도', '시부모', '대가족', '노', '거진', '엄두', '끼', '습', 
                        '야무', '통돌', '힙', '하스', '캣', '브리즈', '다면', '서실', '센말', '하루살이', '덱', '위서', '강학', '아차', '본디', '반전', '란', '굴레', '유레이', '카이', '카이런', '칸칸', '새시', '원츄', '필두', '쎄', '이운', '지랄', 
                        '시누이', '샹', '편당', '연함', '부지런', '타보', '거스', '과한', '힝', '추기', '쌔', '삥', '몫', '흔상', '장안', '탈거', '야옹', '제방', '전번', '긴대', '모친', '연연', '복락', '엉거', '매요', '이호', '막이', '맞교환', '락킹', 
                        '엥', '세규', '우선시', '누님', '대소사', '배송굿', '준거', '정력', '래서', '원품', '송가', '월임', '짬', '달집', '색원', '보아', '우왕', '주남', '솔드', '빌립', '운지', '사질', '상상외', '뷰', '무진장', '사연', '별생각', '갇', 
                        '륭', '루미', '매몰', '루만', '쥔장', '씽', '리조', '단부', '혁기', '완저', '요마', '흠결', '평이', '삼파전', '맨', '밀립', '사라', '집진기', '다영', '취라', '연공', '웰러', '으', '킹왕짱', '안조', '꽝', '유용하', '이면', '뽁', 
                        '없음', '보단', '만당', '이라니', '거조', '이타', '별것', '매장이', '돌리지', '웬일', '킵'
                        
                        # maskpack 추가
                        '여깁니다', '밤사이','텐데요','나머진', '이걸로','김성윤','가규','리쥬','처','전반','리더스더마픽스','더블유','베토벤','엄청','잔느식스헬로케어팩','액비','이모','아저씨','아비브','덩어리','자음생','조리원','기분전환','사보','메르비',
                        '사시','인감','특정','눈앞','아들딸','중반','아리디','딸애','눈두덩','편','탈','한숨','거짓말','순수크린','직전','생김','협찬','닥터웰메이드','서우콜라겐','히터','서우','골드핏마스크','선호','유학길','목표','자취','게으름뱅이','야대',
                        '점도','선두','리머','올해','미래','능','일향','일대','뭉텅이','한상','업무상','대면','취업','계획','무지한','불혹','면에','자용','부부','엡','핍','겇','올케언니','별말','남학생','무공','서함','들통','로로스키니','다회차','내용품',
                        '봄여름','슥','자부','원위치','수정','님','셀카','핑','디엠','해선','용재','확대','사모','조아지','경찹','서랍','살점','발해','런지','테앵','잇기','페리홉','친언니','연습','하녀','코이','코나','한나절','마리오네트','무척','그럽','요즈음',
                        '예랑','심부','낼','소꿉친구','미련','헤거','예인','알가','메트','리콜','케비아','에르메스','실감','약감','활','사바','과도','태한','부지','프로셀','쐐','아오','쯤','맡','렁','후애','자면','아크','짜도','우와','둥','당연','놀','시댁',
                        '피몽','쉐','시어머니','실현','앞전','미남','데카','소사','이드','앙몽','듸','억제','삭','개안','소테','쪼금','기어','개개','골짝','피프','피코','허신','취소','센타','밈','감고','샤프','이만저만','항구','이브','가금','착면','우태',
                        '덕선','선라','사합','키','튜','하나하나','직접','파티션','전전긍긍','챙','거가','러플','접이','탕요','바케','바흐','파하','앆','구매','착시','넉지','율','알뜰','건몰','태산','지사장','임박','적지','주차','바림','지와','바보','다라',
                        '서진','스콜라','나만','식','연애','유정','재현','마이드','아기씨','재구','매입','비부','긷','야근','틈','헬로','고행','쟌느','가농','발처','인심','토털','게유','들보','비티','카구','숭','힌','타애','애너','우아','누우','투타','숙부',
                        '쟁두','무네','이상무','효가','방판','최신','최초','의지','확','악','절반','러너','다로','찬','헵','고영','순','메디'
                        
                        # 레깅스 추가
                        '입', '존', '테스', '반품', '택배', '사둔', '수어', '천도', '물놀', '제기', '편', '녹', '어머님', '쑥국', '시어머니', '오크', '밀', '룰루', '룰', '엘레', '딩', '딸내미', '장',  '깡', '컷', '적리', '네영', '이모', '당',
                        '입진', '통도', '워너', '비즈', '틱', '머서', '주수', '입어', '교환', '불편', '선호', '행복',
                        ]


product['changed_text'] = product['add_rule_based_split_review'].apply(lambda texts : compiled_morphs_noun(texts, stopwords_path, additional_stopwords))

In [ ]:
#pip install openpyxl

In [ ]:
wireless_stop = pd.read_excel('C:/Users/songy/Desktop/Data/wireless_data/wireless_stopword.xlsx')
print(list(wireless_stop['형태소']))

In [ ]:
len(product)

In [ ]:
product.head(3)

In [ ]:
product.head(3)

In [ ]:
def list_convert_comma(lst):
    return ', '.join(lst)

def list_convert_space(lst):
    return ' '.join(lst)

In [ ]:
product['Tokenized_sentence'] = product['changed_text'].apply(list_convert_comma)
product['Tokenized_morphs'] = product['changed_text'].apply(list_convert_space)
product.head(3)

In [ ]:
product.rename(columns={'changed_text':'Tokenized_sentence',
                        'Tokenized_sentence':'Tokenized_sentence_list'}, inplace=True)

In [ ]:
#product.to_csv('C:/Users/songy/Desktop/Data/wireless_product_morphs.csv', encoding='utf-8-sig')

- 앞 전처리 (replace) 부분에서 처리 방식이 상이한 부분이 있어 추후 통일 예정
- 우선 통일화 위해 공유 받은 데이터로 결과값 추출 진행 (product_pos)

In [ ]:
product

In [ ]:
# 위 함수 적용
#product['Tokenized_sentence'] = product['Tokenized_sentence'].apply(str_to_list)

In [ ]:
remove_stopwords2 = sum(product['Tokenized_sentence'],[])
remove_stopwords2

In [ ]:
#remove_stopwords2 = sum(product['changed_text'],[])
#remove_stopwords2

In [ ]:
from collections import Counter
remain_freq = Counter(remove_stopwords2).most_common()
df_remain_freq = pd.DataFrame(remain_freq, columns=['형태소', '빈도'])
df_remain_freq

In [ ]:
from pprint import pprint
print(df_remain_freq)

In [ ]:
df_remain_freq.to_csv('C:/Users/songy/Desktop/Data/maskpack_data/mask_product_keyword.csv', encoding = 'utf-8-sig')

In [ ]:
df_remain_freq

In [ ]:
#df_remain_freq['빈도'].value_counts().to_frame().reset_index().sort_values(by='빈도', ascending=False)[:30]

In [ ]:
plt.rc("font", family = "Malgun Gothic")
sns.set(font="Malgun Gothic", 
rc={"axes.unicode_minus":False}, style='white')

In [ ]:
import seaborn as sns
plt.rc('font', size = 20)
ax = plt.subplots(figsize=(25,10))
plt.xticks(rotation=90)
sns.barplot(data=df_remain_freq[:70], x='형태소', y='빈도')

- elbow 기반 : 어댑터까지

In [ ]:
#df_remain_freq['퍼센트'] = round((df_remain_freq['빈도']/df_remain_freq['빈도'].sum())*100,2)
#df_remain_freq['cumsum'] = df_remain_freq['퍼센트'].cumsum()
#df_remain_freq

In [ ]:
df_remain_freq

In [ ]:
for i in range(5):
    print(df_remain_freq['빈도'][i])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_pos_tag(word):
    "명사만 추출하기 위해"
    if isinstance(word, str):  # 입력이 문자열인지 확인
        pos_tags = m.pos(word)
        return pos_tags[0][1] if pos_tags else 'Unknown'
    else:
        return 'Unknown'

df_remain_freq['pos'] = df_remain_freq['형태소'].apply(get_pos_tag)
df_remain_freq.head(3)

In [ ]:
df_remain_freq['pos'].value_counts().to_frame().reset_index()

In [ ]:
# Test
#df_remain_freq[(df_remain_freq['pos']=='NNG') | (df_remain_freq['pos']=='NNP')].reset_index()[['형태소','빈도','pos']]

## 방법론2 > H0: 비슷한 속성이 많을수록 중요한 속성이다. 

- 방법론1과 다르게 자기 자신을 포함해 진행 (제외 부분 제거 필요)

In [ ]:
# 명사만 추출하는 부분
df_remain_freq = df_remain_freq[(df_remain_freq['pos']=='NNG') | (df_remain_freq['pos']=='NNP')].reset_index()[['형태소','빈도','pos']]

# 변화량 및 변화량/총변화량 도출 부분
df_remain_freq['변화량'] = 0

for i in range(len(df_remain_freq)):
    if i == 0:
        df_remain_freq['변화량'][i] = 0
    else:
        # ascending 되어 있어 증가하는 것은 없기 때문에 제곱근은 배제 
        df_remain_freq['변화량'][i] = df_remain_freq['빈도'][i-1] - df_remain_freq['빈도'][i]
    
    
    changed_sum = df_remain_freq['변화량'].sum()

# 퍼센트 계산으로 100을 곱하는 것 > 의미 있는 결과를 위해 round를 먼저 취해주고 100 곱함 
df_remain_freq['변화량/총변화량'] =round((df_remain_freq['변화량']/changed_sum),2)*100

# 두번째 minima 찾는 부분 (변화량/총변화량 = 0인 두번째 부분)
# 0행이 0으로 시작함에 따라 2번째 변곡점인 2로 진행
second_zero_index = df_remain_freq[df_remain_freq['변화량/총변화량']==0].index[2]
x_labels = df_remain_freq['형태소'][:second_zero_index+10]
print(f'minima is : {x_labels[second_zero_index]}, {second_zero_index}번째')

# 그래프 부분
fig1 = plt.figure(figsize=(25,8))
fig1.gca().spines['right'].set_visible(False)   # 오른쪽 테두리 제거
fig1.gca().spines['top'].set_visible(False)     # 위 테두리 제거
area1 = fig1.add_subplot(1,1,1)
ax1 = sns.lineplot(df_remain_freq['변화량/총변화량'][:second_zero_index+10], linewidth=3, ax=area1)   # ax=area1
plt.axvline(x=df_remain_freq[df_remain_freq['변화량/총변화량']==0.00].index[1], color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
plt.axvline(x=second_zero_index, color='r', linestyle='--', linewidth=1.5, alpha=0.7)
plt.xticks(ticks=range(len(x_labels)), labels=x_labels, rotation=90)
ax1.set_title('Finding Minima', fontsize=24)    #position=(0.5, 5)

In [ ]:
df_remain_freq

In [ ]:
#df_remain_freq[df_remain_freq['변화량/총변화량']==0]

### 키워드별 속성 추출 알고리즘 구축 

In [ ]:
product.head(3)

In [ ]:
# 마지막에 해당 코드 코드화하기 (누적 퍼센트, TopN개 파라미터 포함)
exclude_list = []   # 여기엔 앞에 등장한 키워드들 10개 포함해 누적 저장됨
list_all = []
subset_list = []

for i in range(second_zero_index):
    # A : 형태소 키워드 index 5번 (어댑터)까지 확인 
    # i번째 형태소가 명사에 해당할 경우만 

    # 동적 변수명 생성해 데이터프레임 구성 
    morphs_target = df_remain_freq['형태소'][i]
    var_name = f'df_{morphs_target}'
    globals()[var_name] = product[product['add_rule_based_split_review'].str.contains(f'{df_remain_freq["형태소"][i]}')]


    # 키워드별 형태소 키워드 분석
    var_name2 = f'keyword_freq_{morphs_target}'
    globals()[var_name2] = pd.DataFrame(Counter(sum(globals()[var_name]['Tokenized_sentence'],[])).most_common(), columns=['형태소', '빈도'])
    ### 앞단 키워드 상위 10개 제외 리스트 저장 (회의 반영 : 중복과정 제거)
    ### 제외한 키워드들만 형태소 feature에 반영될 수 있게, 퍼센트 뽑기 전에 키워드 루프

    globals()[var_name2] = globals()[var_name2][~globals()[var_name2]['형태소'].isin(exclude_list)]
    #exclude_list.extend(list(globals()[var_name2]['형태소'][:10]))
    exclude_list.extend(list(globals()[var_name2]['형태소'][:0]))

    #print(exclude_list)

    # 자기 자신은 제외하는 부분 (순서 변경)
    #globals()[var_name2] = globals()[var_name2][globals()[var_name2]['형태소']!=morphs_target]
        
    # 퍼센트, 누적 퍼센트 feature 추가 
    globals()[var_name2]['퍼센트'] = round((globals()[var_name2]['빈도']/globals()[var_name2]['빈도'].sum())*100,2)
    globals()[var_name2]['누적_퍼센트'] = globals()[var_name2]['퍼센트'].cumsum()

    # 누적 퍼센트에 따른 키워드 데이터프레임 추출
    c_value = 70                                        # 누적_퍼센트 파라미터 설정 / 누적 파라미터 이하인 데이터만 추출됨 
    var_name3 = f'cumsum_keyword_{morphs_target}'
    #globals()[var_name3] = globals()[var_name2][(globals()[var_name2]['누적_퍼센트']<c_value)]  # 자기 자신 제외 부분 앞단 (globals()[var_name2]['형태소']!=morphs_target) >> 방법론2에서는 제거
    globals()[var_name3] = globals()[var_name2][(globals()[var_name2]['누적_퍼센트']<c_value)]  # 자기 자신 제외 부분 앞단 (globals()[var_name2]['형태소']!=morphs_target) >> 방법론2에서는 제거

    # subset set으로 저장 (추후 겹치는 퍼센트에 따라 적합성 판단)
    var_name4 = f'subset_{morphs_target}'
    globals()[var_name4] = set(globals()[var_name3]['형태소'])

    # 전체 리스트 확인 & 전체 리스트가 전체 형태소중에 얼만큼의 비율을 차지하는지 
    list_all.extend(list(globals()[var_name3]['형태소']))
    list_set = set(list_all)
    #print(f'{i}번째 리스트 : \n{list_all}\n')

    #### subset끼리 계산 (i개씩 비교)
    subset_list.append(var_name4)       # 루프당 존재하는 subset 집합
    print(subset_list)
    subset_number = len(subset_list)    # subset 개수

#print(subset_list)
print('')

# print(list_all) # 전체 형태소 뽑고 싶을 때 
print(f'세부 고유 키워드가 전체 형태소 결과 포함 비율 : {round((len(list_set) / len(df_remain_freq.형태소))*100,2)} %')

sub_result_sum = df_remain_freq[df_remain_freq['형태소'].isin(list_all)]['빈도'].sum()
all_result_sum = df_remain_freq['빈도'].sum()
print(f'세부 키워드가 전체 키워드 내 차지하는 비율    : {round((sub_result_sum/all_result_sum)*100,2)} %')

print(f'\n현재 존재하는 subset의 개수는 {subset_number}')

In [ ]:
import itertools

def get_set_by_name(name):
    return globals()[name]

def find_intersections(sets_names):
    sets = [get_set_by_name(name) for name in sets_names]
    n = len(sets)
    results = []

    # 모든 가능한 부분집합에 대해 겹치는 요소 찾기
    for r in range(n + 1):
        for combo in itertools.combinations(range(n), r):
            if combo:  # 공집합은 생략
                intersection = sets[combo[0]]
                for i in combo[1:]:
                    intersection = intersection.intersection(sets[i])
                results.append({"Set Combination": tuple(sets_names[i] for i in combo), "Intersection": intersection})
            else:
                results.append({"Set Combination": combo, "Intersection": set()})
    return results

def intersection_to_df(sets_names):
    intersections = find_intersections(sets_names)
    # 데이터프레임으로 변환
    df = pd.DataFrame(intersections)
    # 결과 출력
    return df

def count_subset_num(sets):
    return len(sets)

# 필터링 되는지 확인
#df_sample[df_sample['Intersection'].apply(lambda x: len(x) < 1)]['Intersection'].value_counts()

# 예시 실행
df_sample = intersection_to_df(subset_list)
df_sample['set_number'] = df_sample['Set Combination'].apply(count_subset_num)
df_sample = df_sample[df_sample['Intersection'].apply(lambda x: len(x) > 1)]
df_sample

In [ ]:
df_sample.to_csv('C:/Users/songy/Desktop/Data/combination_result_240806.csv', encoding='utf-8-sig')

In [ ]:
subset_list

In [ ]:
df_sample['Intersection'] = df_sample['Intersection'].apply(lambda x: list(x))

In [ ]:
def add_quotes(value):
    return f'"{value}"'

In [ ]:
#df_sample['Intersection'] = df_sample['Intersection'].apply(lambda x: f'{x}')
#df_sample.head()

In [ ]:
def percent_subset(list_name):
    x = df_remain_freq[df_remain_freq['형태소'].isin(list_name)]
    x2 = (x['빈도'].sum() / df_remain_freq['빈도'].sum())*100
    x2 = round(x2, 2)
    return x2

In [ ]:
#df_sample.to_csv('C:/Users/songy/Desktop/Data/result_way2.csv', encoding='utf-8-sig')

In [ ]:
df_sample['word_percent'] = df_sample['Intersection'].apply(percent_subset)
df_sample.head()

In [ ]:
df_sample[df_sample['set_number'] == 1]

In [ ]:
set_count_df = df_sample['set_number'].value_counts().to_frame().reset_index().sort_values(by='set_number', ascending=True)
set_count_df

In [ ]:
sns.barplot(set_count_df, x='set_number', y='count')

In [ ]:
sns.scatterplot(data=df_sample, x='set_number', y='word_percent', hue='set_number')

In [ ]:
# 추후 합계까지 함수화 가능 할 듯 
print(f'1번 겹치는 경우 : 전체 경우의 {round(((df_sample[df_sample.set_number == 1].word_percent.sum())/(len(df_sample)*100))*100,2)} %')
print(f'2번 겹치는 경우 : 전체 경우의 {round(((df_sample[df_sample.set_number == 2].word_percent.sum())/(len(df_sample)*100))*100,2)} %')
print(f'3번 겹치는 경우 : 전체 경우의 {round(((df_sample[df_sample.set_number == 3].word_percent.sum())/(len(df_sample)*100))*100,2)} %')
print(f'4번 겹치는 경우 : 전체 경우의 {round(((df_sample[df_sample.set_number == 4].word_percent.sum())/(len(df_sample)*100))*100,2)} %')
print(f'5번 겹치는 경우 : 전체 경우의 {round(((df_sample[df_sample.set_number == 5].word_percent.sum())/(len(df_sample)*100))*100,2)} %')
print(f'6번 겹치는 경우 : 전체 경우의 {round(((df_sample[df_sample.set_number == 6].word_percent.sum())/(len(df_sample)*100))*100,2)} %')

In [ ]:
df_sample[(df_sample['set_number']==6)]

In [ ]:
def check_not_in_intersection(row):
    "subset_ 이 없는 경우 False로 반환"
    set_combination = row['Set Combination']
    intersection_list = row['Intersection']

    not_in_list = []
    for w in set_combination:
        if w.startswith('subset_'):
            subset_word = w[len('subset_'):]
            if subset_word not in intersection_list:
                not_in_list.append(subset_word)

    return not_in_list

df_sample['not_in_intersection'] = df_sample.apply(check_not_in_intersection, axis=1)
df_sample.head(3)

In [ ]:
df_sample['concated_word_list'] = df_sample['Intersection'] + df_sample['not_in_intersection']
df_sample.head()

In [ ]:
df_sample['word_percent_concated'] = df_sample['concated_word_list'].apply(percent_subset)
df_sample.head()

In [ ]:
sns.scatterplot(data=df_sample, x='set_number', y='word_percent_concated', hue='set_number')

In [ ]:
df_sample.head(3)

In [ ]:
df_remain_freq['빈도'].sum()

In [ ]:
df_remain_freq

In [ ]:
df_remain_freq.to_csv('C:/Users/songy/Desktop/Data/pad_data/pad_define_stopwords.csv',encoding='utf-8-sig')

In [ ]:
"""
def word_percent_fuction(df_remain_freq, subset):
    "subset 안 요소 있는 행 추출하고, 빈도의 합 / 전체 빈도의 합"
    df_remain_freq['형태소'].
"""

In [ ]:
df_sample.head(3)

In [ ]:
"""
for combination in df_sample['Set Combination']:
    #print(combination)
    print(float(df_sample[df_sample['Set Combination']==combination]['word_percent']))
"""

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
import ast

# Set Combination에 들어있는 고유 집합들을 unqique_subsets에 저장
unique_subsets = set()
for combination in df_sample['Set Combination']:
    unique_subsets.update(eval(combination) if isinstance(combination, str) else combination)

unique_subsets = list(unique_subsets)
#print(unique_subsets)

# 집합(행)별로 set combination 포함 여부에 따라 1 또는 0으로 저장
binary_matrix = []
for subset in unique_subsets:
    row = []
    word_percent = []
    for combination in df_sample['Set Combination']:
        combination = eval(combination) if isinstance(combination, str) else combination  
        #row.append(1 if subset in combination else 0)
        # percent로 삽입하는 부분
        word_percent.append(float(df_sample[df_sample['Set Combination']==combination]['word_percent']/100) if subset in combination else 0)  
    #binary_matrix.append(row)
    binary_matrix.append(word_percent)

# combination 포함 결과 numpy 저장
binary_matrix = np.array(binary_matrix)

# 계층적 군집화에 combination 포함 결과 반영 수행
Z = linkage(binary_matrix, method='ward')

# 시각화 
plt.figure(figsize=(25, 7))
dendro = dendrogram(Z, labels=unique_subsets, leaf_rotation=90) 
plt.title('Hierarchical Clustering Dendrogram for Subsets')
plt.xlabel('Subsets')
plt.ylabel('Distance')
plt.show()

In [ ]:
#### 노드 색상별 추출하는 방법 
from scipy.cluster.hierarchy import fcluster

# 덴드로그램의 color_list 정보를 사용하여 클러스터링
color_list = dendro['color_list']
icoord = np.array(dendro['icoord'])
dleaf_colors = np.array([dendro['leaves_color_list'][i] for i in range(len(unique_subsets))])
dleaf_labels = np.array(dendro['ivl'])

# 고유 색상 추출
unique_colors = np.unique(dleaf_colors)

# 클러스터별 집합 도출
cluster_dict = {}
for color in unique_colors:
    cluster_dict[color] = dleaf_labels[dleaf_colors == color].tolist()

def subword_total(st):
    "클러스터별 속성을 도출하기 위한 함수"
    set_all = set()
    for i in st:
        set_all.update(globals()[i])
    return set_all

#result_cluster = pd.DataFrame([(k,v) for k, vs in cluster_dict.items() for v in vs], columns=['cluster','set_list'])

# 클러스터별 집합 출력
for color, subsets in cluster_dict.items():
    print(f"Cluster color {color}: {subsets}")
    print(f"결과 속성은 아래와 같습니다. \n{subword_total(subsets)}\n\n")

In [ ]:
cluster_dict

In [ ]:
unique_colors

In [ ]:
result_cluster = pd.DataFrame([(k,v) for k, vs in cluster_dict.items() for v in vs], columns=['cluster','set_list'])
result_cluster = result_cluster.groupby('cluster').apply(lambda x : x.assign(set_list=[cluster_dict[x.name]]*len(x))).reset_index(drop=True).drop_duplicates(['cluster'])
result_cluster['subword'] = result_cluster['set_list'].apply(subword_total)
result_cluster

### BERT / KeyBERT 실험 진행

BERT

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# BERT 모델 및 토크나이저
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
# review 데이터 하나의 Text로 저장
doc_review = ' '.join(product['add_rule_based_split_review'].to_list())
#doc_review

In [ ]:
stopwords_path = "C:/Users/songy/Desktop/Data/stopwords-ko.txt"
with open(stopwords_path, encoding='utf-8') as file:
    stopwords = file.read().splitlines()

stopwords.extend(additional_stopwords)
print(stopwords)

In [ ]:
def get_embedding(text, tokenizer, model):
  inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
  outputs = model(**inputs)
  embedding = outputs.last_hidden_state
  return embedding, inputs

def extract_keywords(text, tokenizer, model, top_n=5):
  # 텍스트에서 명사 추출
  nouns = [word for word in m.nouns(text) if word not in stopwords]
  if len(nouns)==0:
    return []

  # BERT 임베딩 생성
  embedding, inputs = get_embedding(text, tokenizer, model)
  token_embeddings = embedding[0]    # (sequence_length, hidden_size)
  cls_embedding = token_embeddings[0] # [CLS] 토큰 임베딩

  # 토큰 중요도 계산 ([CLS] 토큰과 코사인 유사도)
  token_importance = cosine_similarity(cls_embedding.detach().numpy().reshape(1,-1),
                                       token_embeddings.detach().numpy())

  # 토큰과 중요도 매핑
  tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
  token_scores = list(zip(tokens, token_importance))

  # 상위 N개 키워드 추출
  token_scores = sorted(token_scores, key=lambda x: x[1], reverse=True)
  keywords = [token for token, score in token_scores if token not in ['[CLS]', '[SEP]']][:top_n]

  # 명사만 추출된 키워드 리스트
  keywords = [word for word in keywords if word in nouns]

  return keywords

In [ ]:
extract_keywords(doc_review, tokenizer, model, top_n=5)

KeyBERT

In [ ]:
#!pip install keybert

In [ ]:
from keybert import KeyBERT

# 경량화된 모델로 빠른 연산 속도
kw_model = KeyBERT(model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# distilBERT 기반 더 작은 사이즈의 경량화 모델
kw_distil_model = KeyBERT(model='sentence-transformers/distilbert-base-nli-mean-tokens')

# 다국어 지원 모델
kw_multi_model = KeyBERT(model='sentence-transformers/xlm-r-bert-base-nli-stsb-mean-tokens')

# RoBERTa 기반 문장 임제딩 모델
kw_roberta_model = KeyBERT(model='sentence-transformers/roberta-base-nli-stsb-mean-tokens')

In [ ]:
def keybert_extract_keyword(text, model, top_n=5):
  keywords = model.extract_keywords(text, keyphrase_ngram_range=(1,1), stop_words=stopwords, top_n=top_n)
  return [keyword for keyword, score in keywords]

In [ ]:
def exract_nouns(text, stopwords):
  nouns = [word for word in m.nouns(text) if word not in stopwords]
  return ' '.join(nouns)

In [ ]:
# 명사만 추출 
#noun_text = compiled_morphs_noun(doc_review, stopwords_path, additional_stopwords)
noun_text = exract_nouns(doc_review, stopwords)

In [ ]:
# 키워드 추출
keybert_result = keybert_extract_keyword(noun_text, kw_model, top_n=5)
print(f'keybert (경량화)         result : {keybert_result}')

In [ ]:
keybert_result_doc_list = kw_model.extract_keywords(noun_text,top_n=30)
keybert_df = pd.DataFrame(keybert_result_doc_list, columns=['keyword','score'])
keybert_df

In [ ]:
keybert_distil_result = keybert_extract_keyword(noun_text, kw_distil_model, top_n=5)
print(f'keybert (distil-based)   result : {keybert_distil_result}')

In [ ]:
keybert_multi_result = keybert_extract_keyword(noun_text, kw_multi_model, top_n=5)
print(f'keybert (multi language) result : {keybert_multi_result}')

In [ ]:
keybert_roberta_result = keybert_extract_keyword(noun_text, kw_roberta_model, top_n=5)
print(f'keybert (roberta-based)  result : {keybert_roberta_result}')

In [ ]:
#### top10로 재확인
keybert_result_10 = keybert_extract_keyword(noun_text, kw_model, top_n=10)
print(f'keybert (경량화)         result : {keybert_result_10}')

keybert_distil_result_10 = keybert_extract_keyword(noun_text, kw_distil_model, top_n=10)
print(f'keybert (distil-based)   result : {keybert_distil_result_10}')

keybert_multi_result_10 = keybert_extract_keyword(noun_text, kw_multi_model, top_n=10)
print(f'keybert (multi language) result : {keybert_multi_result_10}')

keybert_roberta_result_10 = keybert_extract_keyword(noun_text, kw_roberta_model, top_n=10)
print(f'keybert (roberta-based)  result : {keybert_roberta_result_10}')

In [ ]:
keybert_distil_result_doc_list = kw_distil_model.extract_keywords(noun_text,top_n=30)
keybert_distil_df = pd.DataFrame(keybert_result_doc_list, columns=['keyword','score'])
keybert_distil_df

In [ ]:
#### top30로 재확인
keybert_result_30 = keybert_extract_keyword(noun_text, kw_model, top_n=30)
print(f'keybert (경량화)         result : {keybert_result_30}')

keybert_distil_result_30 = keybert_extract_keyword(noun_text, kw_distil_model, top_n=30)
print(f'keybert (distil-based)   result : {keybert_distil_result_30}')

keybert_multi_result_30 = keybert_extract_keyword(noun_text, kw_multi_model, top_n=30)
print(f'keybert (multi language) result : {keybert_multi_result_30}')

keybert_roberta_result_30 = keybert_extract_keyword(noun_text, kw_roberta_model, top_n=30)
print(f'keybert (roberta-based)  result : {keybert_roberta_result_30}')

In [ ]:
keybert_multi_result_doc_list = kw_multi_model.extract_keywords(noun_text,top_n=30)
keybert_multi_df = pd.DataFrame(keybert_result_doc_list, columns=['keyword','score'])
keybert_multi_df

In [ ]:
keybert_roberta_result_doc_list = kw_roberta_model.extract_keywords(noun_text,top_n=30)
keybert_roberta_df = pd.DataFrame(keybert_result_doc_list, columns=['keyword','score'])
keybert_roberta_df

### 추가 분석

In [ ]:
# intersection에 대표 속성 포함된 경우
append_keyword_df = df_sample[df_sample['word_percent']!=df_sample['word_percent_concated']]

# intersection에 대표 속성이 포함되지 않은 경우
not_appended_keyword_df = df_sample[df_sample['word_percent']==df_sample['word_percent_concated']]

In [ ]:
append_keyword_df.head(3)

In [ ]:
len(append_keyword_df)

In [ ]:
sns.scatterplot(append_keyword_df, x='word_percent', y='word_percent_concated', hue='set_number')

In [ ]:
sns.scatterplot(not_appended_keyword_df, x='word_percent', y='word_percent_concated', hue='set_number')

In [ ]:
sns.scatterplot(append_keyword_df, x='word_percent', y='word_percent_concated', color='blue', alpha=0.7)
sns.scatterplot(not_appended_keyword_df, x='word_percent', y='word_percent_concated', color='red',alpha=0.6)
plt.legend(labels=['exclude','include'], bbox_to_anchor=(1,1))

In [ ]:
append_keyword_df.head(3)

In [ ]:
# 자기 자신을 포함하지 않는 경우(모든 데이터) 전체 형태소에서 차지하는 비율
df_sample[['set_number','word_percent']].corr()

In [ ]:
# 자기 자신 포함한 경우
df_sample[['set_number','word_percent_concated']].corr()

In [ ]:
not_appended_keyword_df[['set_number','word_percent']].corr()

In [ ]:
append_keyword_df[['set_number','word_percent_concated']].corr()

In [ ]:
append_keyword_df[['set_number','word_percent']].corr()

In [ ]:
s2 = sum(append_keyword_df[append_keyword_df['set_number']==2]['not_in_intersection'],[])
unique_s2 = set(s2)
unique_s2

In [ ]:
print(f"가능     : {s2.count('가능')}")
print(f"고속충전 : {s2.count('고속충전')}")
print(f"멀티탭   : {s2.count('멀티탭')}")
print(f"어댑터   : {s2.count('어댑터')}")
print(f"충전기   : {s2.count('충전기')}")
print(f"콘센트   : {s2.count('콘센트')}")
print(f"포트     : {s2.count('포트')}")

In [ ]:
s3 = sum(append_keyword_df[append_keyword_df['set_number']==3]['not_in_intersection'],[])
unique_s3 = set(s3)
unique_s3

In [ ]:
print(f"가능     : {s3.count('가능')}")
print(f"고속충전 : {s3.count('고속충전')}")
print(f"멀티탭   : {s3.count('멀티탭')}")
print(f"어댑터   : {s3.count('어댑터')}")
print(f"충전기   : {s3.count('충전기')}")
print(f"콘센트   : {s3.count('콘센트')}")
print(f"포트     : {s3.count('포트')}")

In [ ]:
s4 = sum(append_keyword_df[append_keyword_df['set_number']==4]['not_in_intersection'],[])
unique_s4 = set(s4)
unique_s4

In [ ]:
print(f"가능     : {s4.count('가능')}")
print(f"고속충전 : {s4.count('고속충전')}")
print(f"멀티탭   : {s4.count('멀티탭')}")
print(f"어댑터   : {s4.count('어댑터')}")
print(f"충전기   : {s4.count('충전기')}")
print(f"콘센트   : {s4.count('콘센트')}")
print(f"포트     : {s4.count('포트')}")

In [ ]:
s5 = sum(append_keyword_df[append_keyword_df['set_number']==5]['not_in_intersection'],[])
unique_s5 = set(s5)
unique_s5

In [ ]:
print(f"가능     : {s5.count('가능')}")
print(f"고속충전 : {s5.count('고속충전')}")
print(f"멀티탭   : {s5.count('멀티탭')}")
print(f"어댑터   : {s5.count('어댑터')}")
print(f"충전기   : {s5.count('충전기')}")
print(f"콘센트   : {s5.count('콘센트')}")
print(f"포트     : {s5.count('포트')}")

In [ ]:
s6 = sum(append_keyword_df[append_keyword_df['set_number']==6]['not_in_intersection'],[])
unique_s6 = set(s6)
unique_s6

In [ ]:
print(f"가능     : {s6.count('가능')}")
print(f"고속충전 : {s6.count('고속충전')}")
print(f"멀티탭   : {s6.count('멀티탭')}")
print(f"어댑터   : {s6.count('어댑터')}")
print(f"충전기   : {s6.count('충전기')}")
print(f"콘센트   : {s6.count('콘센트')}")
print(f"포트     : {s6.count('포트')}")

In [ ]:
# 대표 속성이 공통 집합에 포함되지 않은 경우 도출되는 키워드 (겹치는 개수별)
append_keyword_df.head()

In [ ]:
def subset_merging_percent(subset_a, subset_b):
    subset_intersection = len(subset_a.intersection(subset_b))
    subset_union = len(subset_a.union(subset_b))
    result = round((subset_intersection / subset_union)*100,2)
    return result

subset_충전
subset_케이스
subset_여행
subset_디자인
subset_어댑터
subset_콘센트
subset_파우치
subset_포트
subset_C타입
subset_가능
subset_고속충전
subset_멀티탭
subset_충전기

In [ ]:
def intersection_list(subset_a, subset_b):
    return subset_a.intersection(subset_b)

In [ ]:
def set_only_list(subset_a, subset_b):
    a_result = subset_a - subset_b
    b_result = subset_b - subset_a
    return print(a_result), print(b_result)